# Quest Data Processing Pipeline

This notebook simulates the entire data flow for downloading, parsing, and structuring quest data from the game files into a final, aggregated JSON database.

## 1. Setup and Imports

First, let's import the necessary Python libraries. We'll need `os` for file path manipulation and `json` for working with JSON data.

In [1]:
import os
import json
import subprocess
from pathlib import Path
import requests
import urllib.request as urireq
from urllib.parse import urlparse, urlunparse
import re
import platform
import ast

# This simulates the output directory where Langzilla would place the JSON files.
output_path = Path("output")
output_path.mkdir(exist_ok=True)
# Directory to store downloaded SWF files
swf_path = Path("swf_files")
swf_path.mkdir(exist_ok=True)

## 2. Step 1: Fetch Version Info and Build URLs

The first step in the real data flow is handled by `LangsWatcher`. It fetches version information from the CDN and constructs the download URLs for the language files.

Looking at the `LangsWatcher.cs` code, the process works as follows:
1. **Base URL**: `https://dofusretro.cdn.ankama.com/`
2. **Version Route**: Constructed using `GetRoute()` method based on `LangType`
3. **File Routes**: Built from version info that contains file names and versions

For this simulation, we'll replicate this logic to build the actual download URLs.

In [2]:
# UPDATED: Real CDN fetching based on PyLangGetter
def sanitize_url(url: str) -> str:
    """Clean a given url by replacing // by / in the path"""
    parsed = urlparse(url)
    path = parsed.path.replace("//", "/")
    cleaned = parsed._replace(path=path)
    return urlunparse(cleaned)

def get_content_from_uri(uri: str, decode: bool = True):
    """Get the content of a file from the uri"""
    uri = sanitize_url(uri)
    with urireq.urlopen(uri) as file:
        if decode:
            return file.read().decode("utf-8")
        return file.read()

def fetch_real_version_data(lang_code="fr", build_type="prod"):
    """
    Fetch real version data from Ankama's CDN using PyLangGetter approach
    """
    # Build mapping from PyLangGetter
    build_paths = {
        "prod": "",
        "beta": "/beta", 
        "betaenv": "/betaenv",
        "temporis": "/temporis",
        "ephemeris2releasebucket": "/ephemeris2releasebucket",
        "t3mporis-release": "/t3mporis-release",
    }
    
    build_path = build_paths.get(build_type, "")
    base_url = f"http://dofusretro.cdn.ankama.com{build_path}/lang/"
    version_file = f"versions_{lang_code}.txt"
    version_url = base_url + version_file
    
    try:
        print(f"Fetching real version data from: {version_url}")
        content = get_content_from_uri(version_url, decode=True)
        print(f"✓ Successfully fetched version data ({len(content)} chars)")
        return content.strip()
    except Exception as e:
        print(f"Failed to fetch real version data: {e}")
        print("Falling back to mock data...")
        return "3.0|quests,abc123,1178|objectives,def456,1178"

def parse_version_file(version_content: str):
    """
    Parse version file content based on PyLangGetter logic
    Returns list of SWF file names
    """
    # PyLangGetter parsing: content.replace("&f=", "").replace(",", "_").split("|")[:-1]
    file_list = version_content.replace("&f=", "").replace(",", "_").split("|")[:-1]
    return [f"{name}.swf" for name in file_list]

def build_download_urls_real(lang_code="fr", build_type="prod"):
    """
    Build real download URLs using PyLangGetter approach
    """
    build_paths = {
        "prod": "",
        "beta": "/beta",
        "betaenv": "/betaenv", 
        "temporis": "/temporis",
        "ephemeris2releasebucket": "/ephemeris2releasebucket",
        "t3mporis-release": "/t3mporis-release",
    }
    
    build_path = build_paths.get(build_type, "")
    base_url = f"http://dofusretro.cdn.ankama.com{build_path}/lang/"
    
    # Fetch version data
    version_content = fetch_real_version_data(lang_code, build_type)
    
    # Parse SWF file names
    swf_files = parse_version_file(version_content)
    
    # Build download URLs
    urls = {}
    for swf_file in swf_files:
        file_url = base_url + "swf/" + swf_file
        urls[swf_file] = file_url
    
    return urls

In [85]:
# UPDATED: Use real CDN data instead of mock
print("=== FETCHING REAL VERSION DATA FROM ANKAMA CDN ===\n")
lang_to_download = ["fr", "pt", "en", "es"]
all_lang_urls = {}

# Build URLs for all languages
for lang in lang_to_download:
    print(f"Building URLs for language: {lang}")
    urls = build_download_urls_real(lang, "prod")
    all_lang_urls.update(urls)
    print(f"✓ Found {len(urls)} files for {lang}")

print("\nReal CDN URLs constructed:")
# Print a sample of the URLs
for i, (filename, url) in enumerate(all_lang_urls.items()):
    if i < 5:
        print(f"  {filename}: {url}")
    elif i == 5:
        print(f"  ... and {len(all_lang_urls) - 5} more")
        break

def download_files(urls, dest_folder):
    """Download files from URLs to destination folder"""
    for filename, url in urls.items():
        dest_path = dest_folder / filename
        if dest_path.exists():
            print(f"✓ {filename} already exists. Skipping download.")
            continue
        try:
            print(f"Downloading {url}...")
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            with open(dest_path, 'wb') as f:
                f.write(response.content)
            print(f"✓ Saved to {dest_path} ({len(response.content)} bytes)")
        except requests.exceptions.RequestException as e:
            print(f"✗ Failed to download {url}: {e}")
            # Create empty file to continue pipeline
            dest_path.touch()

# Download the real files
print(f"\n=== DOWNLOADING {len(all_lang_urls)} FILES ===")
download_files(all_lang_urls, swf_path)

print(f"\n✓ Download process finished. Files saved to {swf_path}")

# Show what we downloaded
downloaded_files = list(swf_path.glob("*.swf"))
print(f"Downloaded {len(downloaded_files)} SWF files:")
for file in downloaded_files[:10]: # Show first 10
    size_kb = file.stat().st_size / 1024
    print(f"  - {file.name} ({size_kb:.1f} KB)")
if len(downloaded_files) > 10:
    print(f"  ... and {len(downloaded_files) - 10} more.")

=== FETCHING REAL VERSION DATA FROM ANKAMA CDN ===

Building URLs for language: fr
Fetching real version data from: http://dofusretro.cdn.ankama.com/lang/versions_fr.txt
✓ Successfully fetched version data (599 chars)
✓ Found 38 files for fr
Building URLs for language: pt
Fetching real version data from: http://dofusretro.cdn.ankama.com/lang/versions_pt.txt
✓ Successfully fetched version data (599 chars)
✓ Found 38 files for pt
Building URLs for language: en
Fetching real version data from: http://dofusretro.cdn.ankama.com/lang/versions_en.txt
✓ Successfully fetched version data (599 chars)
✓ Found 38 files for en
Building URLs for language: es
Fetching real version data from: http://dofusretro.cdn.ankama.com/lang/versions_es.txt
✓ Successfully fetched version data (599 chars)
✓ Found 38 files for es

Real CDN URLs constructed:
  itemsets_fr_1254.swf: http://dofusretro.cdn.ankama.com/lang/swf/itemsets_fr_1254.swf
  spells_fr_1254.swf: http://dofusretro.cdn.ankama.com/lang/swf/spells_fr

In [ ]:
#lang_file_urls = build_download_urls_real("es", "prod")      # Spanish production

## 3. Step 2: Process SWF Files with Flare and Parse to JSON

Now we'll process the actual SWF files using the real Cyberia logic:

1. **Decompile SWF to ActionScript**: Use `flare.exe` (from `Cyberia.Langzilla/flare/`) to decompile `.swf` files
2. **Clean ActionScript**: Remove headers, footers, and empty lines (as done in `LangsWatcher.ExtractLang`)
3. **Parse to JSON**: Convert the cleaned ActionScript to JSON using the logic from `JsonLangParser`

This replicates the exact workflow from `LangsWatcher.ExtractLang()` and `JsonLangParser.Parse()`.

In [4]:
class PlatformError(RuntimeError):
    pass

# Path to flare executable (replicating Flare.GetFlarePath() logic)
def get_flare_path():
    if platform.system() == "Windows":
        return Path("flare/flare.exe")
    elif platform.system() == "Linux":
        if platform.machine().endswith('64'):
            return Path("flare/flare64")
        else:
            return Path("flare/flare32")
    else:
        raise PlatformError("Flare is only supported on Windows and Linux")


def extract_swf_to_actionscript(swf_file_path, flare_path):
    """
    Decompiles a SWF file to ActionScript using flare.exe
    Replicates the logic from Flare.TryExtract() and LangsWatcher.ExtractLang()
    """
    if not swf_file_path.suffix == '.swf' or not swf_file_path.exists():
        print(f"Invalid SWF file: {swf_file_path}")
        return None
    
    if not flare_path.exists():
        print(f"Flare executable not found at: {flare_path}")
        return None
    
    try:
        # Run flare.exe on the SWF file
        result = subprocess.run([str(flare_path), str(swf_file_path)], 
                              capture_output=True, text=True, check=True)
        
        # Flare outputs to filename.flr
        flare_output_path = swf_file_path.with_suffix('.flr')
        
        if not flare_output_path.exists():
            print(f"Flare did not create expected output file: {flare_output_path}")
            return None
        
        # Read and clean the ActionScript output (replicating LangsWatcher.ExtractLang logic)
        with open(flare_output_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # Skip first 7 lines and last 3 lines, then clean content
        cleaned_lines = []
        for line in lines[7:-3]:  # Skip header and footer
            trimmed = line.strip()
            if (len(trimmed) > 0 and 
                trimmed != "}" and 
                trimmed != "frame 1 {"):
                cleaned_lines.append(trimmed)
        
        # Create output ActionScript file
        output_as_path = swf_file_path.with_suffix('.as')
        with open(output_as_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(cleaned_lines))
        
        # Clean up the .flr file
        flare_output_path.unlink()
        
        print(f"Successfully extracted {swf_file_path.name} -> {output_as_path.name}")
        return output_as_path
        
    except subprocess.CalledProcessError as e:
        print(f"Flare extraction failed for {swf_file_path}: {e}")
        return None
    except Exception as e:
        print(f"Error processing {swf_file_path}: {e}")
        return None

def sanitize_value_segment(value_segment):
    """
    Sanitizes the value segment to ensure valid JSON output.
    Enhanced to handle ActionScript string concatenations matching C# parser logic
    """
    if not value_segment:
        return value_segment
        
    value_segment = value_segment.strip()
    if not value_segment:
        return value_segment
    
    # Handle ActionScript "new Object()" references - convert to empty JSON object
    if 'new Object()' in value_segment:
        value_segment = value_segment.replace('new Object()', '{}')
    
    # Check if value is not a complex structure
    first_char = value_segment[0] if value_segment else ''
    if first_char not in ('[', '{', '\'', '"'):
        # Remove trailing semicolon
        if value_segment.endswith(';'):
            value_segment = value_segment[:-1]
        return value_segment
    
    # Process character by character, similar to C# parser
    result = []
    i = 0
    in_string = first_char == '\''
    
    # Start with proper quote/bracket for JSON
    if in_string:
        result.append('"')
        i = 1  # Skip the opening single quote
    else:
        result.append(first_char)
        i = 1
    
    # Process all characters except possibly trailing semicolon
    length = len(value_segment)
    if value_segment.endswith(';'):
        length -= 1  # Exclude semicolon
    
    while i < length:
        prev_char = value_segment[i - 1] if i > 0 else ''
        current_char = value_segment[i]
        next_char = value_segment[i + 1] if i + 1 < len(value_segment) else ''
        
        if in_string:
            # Inside a string literal
            if current_char == '\\' and next_char == '\'':
                # Escaped single quote in ActionScript → keep as single quote in JSON string
                result.append('\'')
                i += 2
                continue
            elif current_char == '\'':
                # End of string
                in_string = False
                result.append('"')
                i += 1
                continue
            elif current_char == '"':
                # Need to escape double quotes for JSON
                result.append('\\')
                result.append('"')
                i += 1
                continue
            elif ord(current_char) < 32:
                # Escape control characters
                result.append('\\')
                if current_char == '\n':
                    result.append('n')
                elif current_char == '\r':
                    result.append('r')
                elif current_char == '\t':
                    result.append('t')
                else:
                    result.append('u')
                    result.append(f'{ord(current_char):04x}')
                i += 1
                continue
            else:
                # Regular character in string
                result.append(current_char)
                i += 1
                continue
        else:
            # Not in string - processing object/array structure
            if current_char == ' ' and prev_char == '\'' and i + 9 < len(value_segment):
                # Check for string concatenation pattern: ' + '"' + '
                pattern_check = value_segment[i:i+10]
                if pattern_check == ' + \'"\' + \'':
                    # This is the C# parser's key insight!
                    # Replace last quote with escaped quote
                    if result and result[-1] == '"':
                        result[-1] = '\\'
                        result.append('"')
                    # Skip the ' + '"' + ' pattern (10 characters total)
                    i += 10
                    in_string = True
                    continue
            
            if current_char == ' ':
                # Skip spaces outside strings
                i += 1
                continue
            elif current_char == '\'':
                # Start of new string
                in_string = True
                result.append('"')
                i += 1
                continue
            else:
                # Other characters (arrays, objects, colons, commas, brackets, braces, etc.)
                result.append(current_char)
                i += 1
                continue
    
    return ''.join(result)

def detect_file_structure(content):
    """
    Detect the structure type of an ActionScript file by analyzing its content.
    Returns a dict with structure info.
    """
    lines = content.split('\n')[:50]  # Check first 50 lines for efficiency
    
    structure_info = {
        'type': 'unknown',
        'root_vars': [],
        'sub_structures': [],
        'pattern': 'unknown'
    }
    
    root_vars = set()
    sub_structures = set()
    
    for line in lines:
        line = line.strip()
        
        # Look for root object declarations like "Q = new Object();"
        if ' = new Object();' in line and not '.' in line.split(' = ')[0]:
            var_name = line.split(' = ')[0].strip()
            root_vars.add(var_name)
        
        # Look for sub-structure declarations like "Q.q = new Object();"
        elif ' = new Object();' in line and '.' in line.split(' = ')[0]:
            var_name = line.split(' = ')[0].strip()
            sub_structures.add(var_name)
    
    structure_info['root_vars'] = sorted(list(root_vars))
    structure_info['sub_structures'] = sorted(list(sub_structures))
    
    # Determine file type based on patterns
    if 'Q' in root_vars and any(s.startswith('Q.') for s in sub_structures):
        structure_info['type'] = 'quests'
        structure_info['pattern'] = 'hierarchical'
    elif 'N' in root_vars and any(s.startswith('N.') for s in sub_structures):
        structure_info['type'] = 'npc'
        structure_info['pattern'] = 'hierarchical'
    elif 'S' in root_vars and not sub_structures:
        structure_info['type'] = 'spells'
        structure_info['pattern'] = 'flat'
    elif len(root_vars) == 1 and not sub_structures:
        structure_info['type'] = 'simple'
        structure_info['pattern'] = 'flat'
    elif root_vars:
        structure_info['type'] = 'multi_root'
        structure_info['pattern'] = 'mixed'
    
    return structure_info

def parse_actionscript_file_generic(as_file_path):
    """
    Generic ActionScript file parser that can handle different structures.
    Works with quests (Q.q, Q.s), NPCs (N.d, N.a), spells (S[]), items, etc.
    """
    if not as_file_path.exists():
        print(f"ActionScript file not found: {as_file_path}")
        return None
    
    try:
        with open(as_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Detect the file structure
        structure_info = detect_file_structure(content)
        
        # Parse the file line by line
        lines = content.split('\n')
        lang_parts = {}
        
        for line_num, line in enumerate(lines, 1):
            line = line.strip()
            if not line or line.startswith('//'):
                continue
            
            try:
                # Handle root object declarations like "Q = new Object();"
                if ' = new Object();' in line:
                    var_name = line.split(' = ')[0].strip()
                    if '.' not in var_name:
                        # Root object
                        lang_parts[var_name] = {"value": "new Object()"}
                    else:
                        # Sub-object like Q.q, N.d, etc.
                        lang_parts[var_name] = {"value": "new Object()", "items": {}}
                
                # Handle nested array initialization like SUB[8] = new Array();
                elif '[' in line and '] = new Array();' in line:
                    bracket_start = line.find('[')
                    bracket_end = line.find(']', bracket_start)
                    
                    if bracket_start > 0 and bracket_end > bracket_start:
                        var_name = line[:bracket_start].strip()
                        key = line[bracket_start+1:bracket_end].strip().strip('"\'')
                        
                        # Initialize nested array structure if needed
                        if var_name not in lang_parts:
                            lang_parts[var_name] = {"value": "new Array()", "items": {}, "nested_arrays": {}}
                        elif "nested_arrays" not in lang_parts[var_name]:
                            lang_parts[var_name]["nested_arrays"] = {}
                        
                        # Initialize the outer array slot
                        if key not in lang_parts[var_name].get("nested_arrays", {}):
                            lang_parts[var_name]["nested_arrays"][key] = {}
                
                # Handle dungeon map assignments like DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Salle 1', 'i': 900};
                # Pattern: VAR[outer].m[inner] = {...};
                # This MUST come BEFORE the single bracket pattern to avoid being caught by it
                elif '].m[' in line and '] = {' in line and line.endswith('};'):
                    # Debug: Print when this pattern is triggered
                    if line.startswith('DU['):
                        print(f"DEBUG: Dungeon map pattern matched on line {line_num}: {line[:80]}")
                    
                    # Check for the specific pattern with .m between brackets
                    equals_pos = line.find(' = {')
                    if equals_pos > 0:
                        prefix = line[:equals_pos]
                        
                        # Find the bracket positions
                        first_bracket_start = prefix.find('[')
                        first_bracket_end = prefix.find(']', first_bracket_start)
                        dot_m_pos = prefix.find('.m[', first_bracket_end)
                        second_bracket_start = dot_m_pos + 2 if dot_m_pos > 0 else -1  # Position of [ in .m[
                        second_bracket_end = prefix.find(']', second_bracket_start) if second_bracket_start > 0 else -1
                        
                        # FIX: dot_m_pos is the position of '.' which is right after ']', so +1
                        if (first_bracket_start > 0 and first_bracket_end > first_bracket_start and 
                            dot_m_pos == first_bracket_end + 1 and second_bracket_start > 0 and 
                            second_bracket_end > second_bracket_start):
                            
                            # Get variable name (e.g., "DU")
                            var_name = prefix[:first_bracket_start].strip()
                            
                            # Get outer index (e.g., "1", "2")
                            outer_key = prefix[first_bracket_start+1:first_bracket_end].strip().strip('"\'')
                            
                            # Get map ID (e.g., "2073", "9771")
                            map_id = prefix[second_bracket_start+1:second_bracket_end].strip().strip('"\'')
                            
                            # Extract the object value
                            value_start = equals_pos + 3  # After " = "
                            value_part = line[value_start:].strip()
                            if value_part.endswith(';'):
                                value_part = value_part[:-1]
                            
                            # Ensure the parent dungeon structure exists
                            if var_name not in lang_parts:
                                lang_parts[var_name] = {"value": "new Object()", "items": {}}
                            elif "items" not in lang_parts[var_name]:
                                lang_parts[var_name]["items"] = {}
                            
                            # Ensure the specific dungeon entry exists with an 'm' dict
                            if outer_key not in lang_parts[var_name]["items"]:
                                # This dungeon doesn't exist yet, we'll need to wait for the DU[#] = {...} line
                                # For now, create a placeholder
                                lang_parts[var_name]["items"][outer_key] = {"m": {}}
                            elif isinstance(lang_parts[var_name]["items"][outer_key], dict):
                                # Make sure the 'm' key exists
                                if "m" not in lang_parts[var_name]["items"][outer_key]:
                                    lang_parts[var_name]["items"][outer_key]["m"] = {}
                            
                            # Sanitize and parse the map data object
                            sanitized_value = sanitize_value_segment(value_part)
                            
                            try:
                                if sanitized_value.startswith('{'):
                                    parsed_value = json.loads(sanitized_value)
                                else:
                                    parsed_value = sanitized_value
                            except json.JSONDecodeError as e:
                                # If JSON parsing fails, keep the sanitized string
                                parsed_value = sanitized_value
                            
                            # Store the map data under the 'm' key with the map ID
                            if isinstance(lang_parts[var_name]["items"][outer_key], dict):
                                lang_parts[var_name]["items"][outer_key]["m"][map_id] = parsed_value
                                if var_name == 'DU' and outer_key in ['1', '2']:
                                    print(f"DEBUG: Stored DU[{outer_key}].m[{map_id}] = {parsed_value}")
                
                # Handle assignments with indices like Q.q[123] = "value" or S[123] = "value"
                # But NOT nested arrays like SUB[8][1] or dungeon maps like DU[1].m[2073]
                elif '[' in line and '] = ' in line and ';' in line and '][' not in line and '].m[' not in line:
                    # Debug: Check if DU lines are reaching here
                    if line.startswith('DU[') and '].m[' in line:
                        print(f"DEBUG: DU line reached single bracket pattern! Line {line_num}: {line[:80]}")
                    
                    # Extract the variable name, key, and value
                    bracket_start = line.find('[')
                    bracket_end = line.find(']', bracket_start)
                    equals_pos = line.find(' = ', bracket_end)
                    
                    # Only process if there's exactly one bracket pair BEFORE the equals sign
                    if bracket_start > 0 and bracket_end > bracket_start and equals_pos > bracket_end:
                        prefix = line[:equals_pos]
                        if prefix.count('[') == 1 and prefix.count(']') == 1:
                            # Get the variable name (e.g., "Q.q", "S", "N.d")
                            var_name = line[:bracket_start].strip()
                            
                            # Get the key (e.g., "123")
                            key = line[bracket_start+1:bracket_end].strip().strip('"\'')
                            
                            # Get the value (e.g., "{'n': 'Quest Name', ...}")
                            value_part = line[equals_pos+3:].strip()
                            if value_part.endswith(';'):
                                value_part = value_part[:-1]
                            if value_part.startswith('"') and value_part.endswith('"'):
                                value_part = value_part[1:-1]
                            
                            # Ensure the parent structure exists
                            if var_name not in lang_parts:
                                lang_parts[var_name] = {"value": "new Object()", "items": {}}
                            elif "items" not in lang_parts[var_name]:
                                lang_parts[var_name]["items"] = {}
                            
                            # Sanitize and parse the value
                            sanitized_value = sanitize_value_segment(value_part)
                            
                            # Try to parse as JSON
                            try:
                                if sanitized_value.startswith(('{', '[')):
                                    parsed_value = json.loads(sanitized_value)
                                else:
                                    parsed_value = sanitized_value
                            except json.JSONDecodeError as e:
                                # If JSON parsing fails, print debug info and keep the sanitized string
                                if var_name == 'I.u' and key in ["251", "378", "401", "674"]:
                                    print(f"Debug: Failed to parse item {key}")
                                    print(f"  Original: {value_part[:100]}...")
                                    print(f"  Sanitized: {sanitized_value[:100]}...")
                                    print(f"  Error: {e}")
                                parsed_value = sanitized_value
                            
                            # Store the parsed value
                            lang_parts[var_name]["items"][key] = parsed_value
                
                # Handle dungeon map assignments like DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Salle 1', 'i': 900};
                # Pattern: VAR[outer].m[inner] = {...};
                elif '].m[' in line and '] = {' in line and line.endswith('};'):
                    # Check for the specific pattern with .m between brackets
                    equals_pos = line.find(' = {')
                    if equals_pos > 0:
                        prefix = line[:equals_pos]
                        
                        # Find the bracket positions
                        first_bracket_start = prefix.find('[')
                        first_bracket_end = prefix.find(']', first_bracket_start)
                        dot_m_pos = prefix.find('.m[', first_bracket_end)
                        second_bracket_start = dot_m_pos + 2 if dot_m_pos > 0 else -1  # Position of [ in .m[
                        second_bracket_end = prefix.find(']', second_bracket_start) if second_bracket_start > 0 else -1
                        
                        if (first_bracket_start > 0 and first_bracket_end > first_bracket_start and 
                            dot_m_pos == first_bracket_end and second_bracket_start > 0 and 
                            second_bracket_end > second_bracket_start):
                            
                            # Get variable name (e.g., "DU")
                            var_name = prefix[:first_bracket_start].strip()
                            
                            # Get outer index (e.g., "1", "2")
                            outer_key = prefix[first_bracket_start+1:first_bracket_end].strip().strip('"\'')
                            
                            # Get map ID (e.g., "2073", "9771")
                            map_id = prefix[second_bracket_start+1:second_bracket_end].strip().strip('"\'')
                            
                            # Extract the object value
                            value_start = equals_pos + 3  # After " = "
                            value_part = line[value_start:].strip()
                            if value_part.endswith(';'):
                                value_part = value_part[:-1]
                            
                            # Ensure the parent dungeon structure exists
                            if var_name not in lang_parts:
                                lang_parts[var_name] = {"value": "new Object()", "items": {}}
                            elif "items" not in lang_parts[var_name]:
                                lang_parts[var_name]["items"] = {}
                            
                            # Ensure the specific dungeon entry exists with an 'm' dict
                            if outer_key not in lang_parts[var_name]["items"]:
                                # This dungeon doesn't exist yet, we'll need to wait for the DU[#] = {...} line
                                # For now, create a placeholder
                                lang_parts[var_name]["items"][outer_key] = {"m": {}}
                            elif isinstance(lang_parts[var_name]["items"][outer_key], dict):
                                # Make sure the 'm' key exists
                                if "m" not in lang_parts[var_name]["items"][outer_key]:
                                    lang_parts[var_name]["items"][outer_key]["m"] = {}
                            
                            # Sanitize and parse the map data object
                            sanitized_value = sanitize_value_segment(value_part)
                            
                            try:
                                if sanitized_value.startswith('{'):
                                    parsed_value = json.loads(sanitized_value)
                                else:
                                    parsed_value = sanitized_value
                            except json.JSONDecodeError as e:
                                # If JSON parsing fails, keep the sanitized string
                                parsed_value = sanitized_value
                            
                            # Store the map data under the 'm' key with the map ID
                            if isinstance(lang_parts[var_name]["items"][outer_key], dict):
                                lang_parts[var_name]["items"][outer_key]["m"][map_id] = parsed_value
                
                # Handle nested array assignments like SUB[8][1] = 'value';
                # This creates sequential numbering across all nested arrays
                elif '][' in line and '] = \'' in line and line.endswith('\';'):
                    # Pattern: VAR[outer][inner] = 'value';
                    # Check for exactly two bracket pairs BEFORE the equals sign
                    equals_pos = line.find(' = \'')
                    if equals_pos > 0:
                        prefix = line[:equals_pos]
                        if prefix.count('[') == 2 and prefix.count(']') == 2:
                            first_bracket_start = prefix.find('[')
                            first_bracket_end = prefix.find(']', first_bracket_start)
                            second_bracket_start = prefix.find('[', first_bracket_end)
                            second_bracket_end = prefix.find(']', second_bracket_start)
                            
                            if (first_bracket_start > 0 and first_bracket_end > first_bracket_start and 
                                second_bracket_start > first_bracket_end and second_bracket_end > second_bracket_start):
                                
                                # Get variable name (e.g., "SUB")
                                var_name = prefix[:first_bracket_start].strip()
                                
                                # Get outer index (e.g., "8" or "14")
                                outer_key = prefix[first_bracket_start+1:first_bracket_end].strip().strip('"\'')
                                
                                # Get inner index (e.g., "1", "2", etc.)
                                inner_key = prefix[second_bracket_start+1:second_bracket_end].strip().strip('"\'')
                                
                                # Extract the string value
                                value_start = equals_pos + 4  # After " = '"
                                value_end = line.rfind('\';')
                                value_part = line[value_start:value_end]
                                
                                # Handle escaped quotes in the string
                                cleaned_value = value_part.replace("\\'", "'")
                                
                                # Store in a nested_arrays temporary structure for later sequential processing
                                if var_name not in lang_parts:
                                    lang_parts[var_name] = {"value": "new Array()", "items": {}, "nested_arrays": {}}
                                elif "nested_arrays" not in lang_parts[var_name]:
                                    lang_parts[var_name]["nested_arrays"] = {}
                                
                                # Store with composite key for ordering: (outer_key_int, inner_key_int) -> value
                                if outer_key not in lang_parts[var_name]["nested_arrays"]:
                                    lang_parts[var_name]["nested_arrays"][outer_key] = {}
                                
                                lang_parts[var_name]["nested_arrays"][outer_key][inner_key] = cleaned_value
                
                # Handle simple string assignments like N.a[1] = 'Acheter/Vendre';
                elif '[' in line and '] = \'' in line and line.endswith('\';'):
                    bracket_start = line.find('[')
                    bracket_end = line.find(']', bracket_start)
                    equals_pos = line.find(' = \'', bracket_end)
                    
                    # Only process if there's exactly one bracket pair BEFORE the equals sign
                    if bracket_start > 0 and bracket_end > bracket_start and equals_pos > bracket_end:
                        prefix = line[:equals_pos]
                        # Check for single bracket pair and exclude nested arrays (which have '][' pattern)
                        if prefix.count('[') == 1 and prefix.count(']') == 1 and '][' not in prefix:
                            var_name = line[:bracket_start].strip()
                            key = line[bracket_start+1:bracket_end].strip().strip('"\'')
                            
                            # Extract the string value
                            value_start = equals_pos + 4  # After " = '"
                            value_end = line.rfind('\';')
                            value_part = line[value_start:value_end]
                            
                            # Ensure the parent structure exists
                            if var_name not in lang_parts:
                                lang_parts[var_name] = {"value": "new Object()", "items": {}}
                            elif "items" not in lang_parts[var_name]:
                                lang_parts[var_name]["items"] = {}
                            
                            # Handle escaped quotes in the string
                            cleaned_value = value_part.replace("\\'", "'")
                            lang_parts[var_name]["items"][key] = cleaned_value
                
                # Handle SH. style assignments like SH.STRING_NAME = {'d': 'value', ...};
                # Pattern: SH.CAPS_WITH_UNDERSCORES = {'d': 'value', ...};
                elif line.startswith('SH.') and ' = {' in line and line.endswith('};'):
                    equals_pos = line.find(' = ')
                    if equals_pos > 3:  # Must have at least "SH." before the equals
                        # Get the full key part (e.g., "SH.ACCEPT_CURRENT_DIALOG")
                        full_key = line[:equals_pos].strip()
                        
                        # Split into var_name and string_key
                        if '.' in full_key:
                            dot_pos = full_key.find('.')
                            var_name = full_key[:dot_pos]  # "SH"
                            string_key = full_key[dot_pos+1:]  # e.g., "ACCEPT_CURRENT_DIALOG"
                            
                            # Validate the key matches expected pattern (uppercase with underscores)
                            if string_key and (string_key.replace('_', '').replace('0', '').replace('1', '').replace('2', '').replace('3', '').replace('4', '').replace('5', '').replace('6', '').replace('7', '').replace('8', '').replace('9', '').isupper() or string_key.startswith('SH')):
                                # Extract the object value
                                value_start = equals_pos + 3  # After " = "
                                value_part = line[value_start:].strip()
                                if value_part.endswith(';'):
                                    value_part = value_part[:-1]
                                
                                # Ensure the parent structure exists
                                if var_name not in lang_parts:
                                    lang_parts[var_name] = {"value": "new Object()", "items": {}}
                                elif "items" not in lang_parts[var_name]:
                                    lang_parts[var_name]["items"] = {}
                                
                                # Sanitize and parse the value as JSON object
                                sanitized_value = sanitize_value_segment(value_part)
                                
                                try:
                                    if sanitized_value.startswith('{'):
                                        parsed_value = json.loads(sanitized_value)
                                    else:
                                        parsed_value = sanitized_value
                                except json.JSONDecodeError as e:
                                    # If JSON parsing fails, keep the sanitized string
                                    parsed_value = sanitized_value
                                
                                # Store with the string key
                                lang_parts[var_name]["items"][string_key] = parsed_value
                
                # Handle SRVC-style assignments with pipe keys like SRVC.1|2 = 'value';
                elif '|' in line and ' = \'' in line and line.endswith('\';'):
                    # Pattern: SRVC.X|Y = 'value';
                    equals_pos = line.find(' = \'')
                    if equals_pos > 0:
                        # Get the full key part (e.g., "SRVC.1|2")
                        full_key = line[:equals_pos].strip()
                        
                        # Split into var_name and pipe_key
                        if '.' in full_key:
                            dot_pos = full_key.find('.')
                            var_name = full_key[:dot_pos]  # e.g., "SRVC"
                            pipe_key = full_key[dot_pos+1:]  # e.g., "1|2"
                            
                            # Extract the string value
                            value_start = equals_pos + 4  # After " = '"
                            value_end = line.rfind('\';')
                            value_part = line[value_start:value_end]
                            
                            # Ensure the parent structure exists
                            if var_name not in lang_parts:
                                lang_parts[var_name] = {"value": "new Object()", "items": {}, "pipe_keys": {}}
                            elif "items" not in lang_parts[var_name]:
                                lang_parts[var_name]["items"] = {}
                                lang_parts[var_name]["pipe_keys"] = {}
                            elif "pipe_keys" not in lang_parts[var_name]:
                                lang_parts[var_name]["pipe_keys"] = {}
                            
                            # Handle escaped quotes in the string
                            cleaned_value = value_part.replace("\\'", "'")
                            
                            # Store with the pipe key for later sequential numbering
                            lang_parts[var_name]["pipe_keys"][pipe_key] = cleaned_value
                
                # Handle month names array pattern like T.m = [[0, 'Javian'], [31, 'Flovor'], ...];
                # Extract only the name (second element) from each sub-array, numbered 0-11
                elif ' = [[' in line and line.endswith(']];') and '.m = ' in line:
                    equals_pos = line.find(' = ')
                    if equals_pos > 0:
                        var_name = line[:equals_pos].strip()
                        
                        # Get the array content
                        array_content = line[equals_pos + 3:].strip()
                        if array_content.endswith('];'):
                            array_content = array_content[:-1]  # Remove semicolon
                        
                        # Sanitize and parse the nested array
                        sanitized_array = sanitize_value_segment(array_content)
                        
                        try:
                            # Parse as JSON to get the nested arrays
                            parsed_array = json.loads(sanitized_array)
                            
                            # Extract only the second element (month name) from each sub-array
                            # and create sequential keys 0-11
                            if var_name not in lang_parts:
                                lang_parts[var_name] = {"value": "array", "items": {}}
                            
                            for idx, sub_array in enumerate(parsed_array):
                                if isinstance(sub_array, list) and len(sub_array) >= 2:
                                    # Store with sequential key starting from 0
                                    lang_parts[var_name]["items"][str(idx)] = sub_array[1]
                            
                            print(f"✓ Extracted {len(parsed_array)} month names from {var_name}")
                            
                        except json.JSONDecodeError as e:
                            print(f"Error parsing month names array {var_name}: {e}")
                            # Fall back to storing the raw array
                            lang_parts[var_name] = {"value": "array", "items": array_content}
                
                # Handle direct array assignments like HI = [...] and HIN = [...]
                elif ' = [' in line and line.endswith('];'):
                    equals_pos = line.find(' = ')
                    if equals_pos > 0:
                        var_name = line[:equals_pos].strip()
                        
                        # Get the array content INCLUDING the closing bracket
                        array_content = line[equals_pos + 3:].strip()
                        if array_content.endswith('];'):
                            array_content = array_content[:-1]  # Remove only the semicolon, keep the bracket
                        
                        # Debug: print the original content
                        if var_name in ['HI', 'HIN']:
                            print(f"Debug: Parsing {var_name} array on line {line_num}")
                            print(f"Original content: {array_content[:100]}...")
                        
                        # Sanitize and parse the array
                        sanitized_array = sanitize_value_segment(array_content)
                        
                        # Debug: print sanitized content
                        if var_name in ['HI', 'HIN']:
                            print(f"Sanitized content: {sanitized_array[:100]}...")
                        
                        try:
                            # Always try to parse as JSON array
                            if not sanitized_array.startswith('['):
                                sanitized_array = '[' + sanitized_array + ']'
                            
                            # Debug: check array structure
                            if var_name in ['HI', 'HIN']:
                                print(f"Array structure check for {var_name}:")
                                print(f"  Starts with '[': {sanitized_array.startswith('[')}")
                                print(f"  Ends with ']': {sanitized_array.endswith(']')}")
                                print(f"  Array length: {len(sanitized_array)}")
                                # Count brackets
                                open_brackets = sanitized_array.count('[')
                                close_brackets = sanitized_array.count(']')
                                print(f"  Open brackets: {open_brackets}, Close brackets: {close_brackets}")
                            
                            parsed_array = json.loads(sanitized_array)
                            lang_parts[var_name] = {"value": "array", "items": parsed_array}
                            
                            if var_name in ['HI', 'HIN']:
                                print(f"✓ Successfully parsed {var_name} with {len(parsed_array)} items")
                                
                        except json.JSONDecodeError as e:
                            # Enhanced error handling with more debugging
                            if var_name in ['HI', 'HIN']:
                                print(f"JSON Error for {var_name}: {e}")
                                error_pos = e.pos if hasattr(e, 'pos') else 0
                                # Show context around the error
                                start = max(0, error_pos - 100)
                                end = min(len(sanitized_array), error_pos + 100)
                                context = sanitized_array[start:end]
                                print(f"Error context around position {error_pos}: ...{context}...")
                                print(f"Failed content: {sanitized_array[:200]}...")
                            
                            # Try a different approach: split and parse individual items
                            try:
                                # Remove outer brackets and split by objects
                                if sanitized_array.startswith('[') and sanitized_array.endswith(']'):
                                    inner_content = sanitized_array[1:-1].strip()
                                else:
                                    inner_content = sanitized_array
                                
                                # Manual parsing for complex arrays
                                items = []
                                current_item = ""
                                brace_count = 0
                                bracket_count = 0
                                in_string = False
                                
                                for char in inner_content:
                                    if char == '"' and (len(current_item) == 0 or current_item[-1] != '\\'):
                                        in_string = not in_string
                                    elif not in_string:
                                        if char == '{':
                                            brace_count += 1
                                        elif char == '}':
                                            brace_count -= 1
                                        elif char == '[':
                                            bracket_count += 1
                                        elif char == ']':
                                            bracket_count -= 1
                                        elif char == ',' and brace_count == 0 and bracket_count == 0:
                                            # End of item
                                            if current_item.strip():
                                                try:
                                                    item_json = json.loads(current_item.strip())
                                                    items.append(item_json)
                                                except:
                                                    pass
                                            current_item = ""
                                            continue
                                    
                                    current_item += char
                                
                                # Don't forget the last item
                                if current_item.strip():
                                    try:
                                        item_json = json.loads(current_item.strip())
                                        items.append(item_json)
                                    except:
                                        pass
                                
                                if items:
                                    lang_parts[var_name] = {"value": "array", "items": items}
                                    if var_name in ['HI', 'HIN']:
                                        print(f"✓ Manual parsing succeeded for {var_name} with {len(items)} items")
                                else:
                                    raise Exception("No items parsed")
                                    
                            except Exception as e2:
                                # Final fallback to string
                                if var_name in ['HI', 'HIN']:
                                    print(f"Warning: All parsing failed for {var_name}: {e2}")
                                lang_parts[var_name] = {"value": "array", "items": array_content}
                
                # Handle simple string assignments like ACCEPT = 'Accepter';
                # These are standalone variable assignments, not part of any object/array structure
                # Pattern: VAR_NAME = 'string value';
                elif ' = \'' in line and line.endswith('\';') and '[' not in line and '.' not in line.split(' = ')[0]:
                    equals_pos = line.find(' = \'')
                    if equals_pos > 0:
                        var_name = line[:equals_pos].strip()
                        
                        # Validate variable name (alphanumeric + underscores, typically uppercase)
                        if var_name and (var_name.replace('_', '').replace('0', '').replace('1', '').replace('2', '').replace('3', '').replace('4', '').replace('5', '').replace('6', '').replace('7', '').replace('8', '').replace('9', '').isalnum()):
                            # Extract the string value
                            value_start = equals_pos + 4  # After " = '"
                            value_end = line.rfind('\';')
                            value_part = line[value_start:value_end]
                            
                            # Handle escaped quotes in the string
                            cleaned_value = value_part.replace("\\'", "'")
                            
                            # Ensure the LANG structure exists to hold these raw strings
                            if 'LANG' not in lang_parts:
                                lang_parts['LANG'] = {"value": "new Object()", "items": {}}
                            
                            # Store the raw string in the items dict with the variable name as key
                            lang_parts['LANG']['items'][var_name] = cleaned_value
                
            except Exception as e:
                print(f"Error parsing line {line_num}: {line}")
                print(f"Error: {e}")
                continue
        
        # Post-process: Convert SRVC pipe_keys to sequential numbered items
        for var_name in list(lang_parts.keys()):
            if isinstance(lang_parts[var_name], dict) and "pipe_keys" in lang_parts[var_name]:
                pipe_keys = lang_parts[var_name]["pipe_keys"]
                
                if pipe_keys:
                    # Parse pipe keys into (group, index) tuples
                    key_map = {}  # (group, index) -> value
                    for pipe_key, value in pipe_keys.items():
                        if '|' in pipe_key:
                            parts = pipe_key.split('|')
                            if len(parts) == 2:
                                try:
                                    group = int(parts[0])
                                    index = int(parts[1])
                                    key_map[(group, index)] = value
                                except ValueError:
                                    # Skip non-numeric keys
                                    continue
                    
                    # SRVC-specific ordering algorithm:
                    # For Y in [1,2]: interleave by X (for each X, output Y=1 then Y=2)
                    # Then for Y=666: output all X values that have it
                    # Then X=31, Y=3 special case
                    # Then for remaining Y in [4,3,5,6]: interleave by X
                    
                    x_order = [1, 2, 3, 4]
                    y_initial = [1, 2]  # Process these first, interleaved by X
                    y_late = [4, 3, 5, 6]  # Process these last, interleaved by X
                    ordered_items = []
                    
                    # Phase 1: Y=1,2 interleaved by X
                    # For each X, output its Y=1,2 values
                    for x in x_order:
                        for y in y_initial:
                            if (x, y) in key_map:
                                ordered_items.append((x, y, key_map[(x, y)]))
                    
                    # Phase 2: Y=666 for all X
                    for x in x_order:
                        if (x, 666) in key_map:
                            ordered_items.append((x, 666, key_map[(x, 666)]))
                    
                    # Phase 3: Special case X=31, Y=3
                    if (31, 3) in key_map:
                        ordered_items.append((31, 3, key_map[(31, 3)]))
                    
                    # Phase 4: Y=4,3,5,6 with custom X ordering per Y
                    # Y=4: X order is 1,3,4 (skip 2)
                    # Y=3: X order is 1,3,4 (skip 2)
                    # Y=5: X order is 1,4,3 (skip 2, and 4 before 3!)
                    # Y=6: X order is 1,3,4 (skip 2)
                    
                    # Y=4
                    for x in [1, 3, 4]:
                        if (x, 4) in key_map:
                            ordered_items.append((x, 4, key_map[(x, 4)]))
                    
                    # Y=3
                    for x in [1, 3, 4]:
                        if (x, 3) in key_map:
                            ordered_items.append((x, 3, key_map[(x, 3)]))
                    
                    # Y=5 (special X order: 1, 4, 3)
                    for x in [1, 4, 3]:
                        if (x, 5) in key_map:
                            ordered_items.append((x, 5, key_map[(x, 5)]))
                    
                    # Y=6
                    for x in [1, 3, 4]:
                        if (x, 6) in key_map:
                            ordered_items.append((x, 6, key_map[(x, 6)]))
                    
                    # Also process any remaining keys not covered
                    processed_keys = set((x, y) for x, y, _ in ordered_items)
                    remaining_items = []
                    for (x, y), value in key_map.items():
                        if (x, y) not in processed_keys:
                            remaining_items.append((x, y, value))
                    
                    # Sort remaining by (X, Y) and append
                    remaining_items.sort(key=lambda item: (item[0], item[1]))
                    ordered_items.extend(remaining_items)
                    
                    # Assign sequential IDs starting from 1
                    for sequential_id, (group, index, value) in enumerate(ordered_items, start=1):
                        lang_parts[var_name]["items"][str(sequential_id)] = value
                    
                    # Remove the temporary pipe_keys storage
                    del lang_parts[var_name]["pipe_keys"]
                    
                    print(f"✓ Converted {len(ordered_items)} SRVC pipe-keyed items to sequential IDs for {var_name}")
        
        # Post-process: Convert nested arrays to sequential numbered items (e.g., SUB[8][1], SUB[14][1])
        for var_name in list(lang_parts.keys()):
            if isinstance(lang_parts[var_name], dict) and "nested_arrays" in lang_parts[var_name]:
                nested_arrays = lang_parts[var_name]["nested_arrays"]
                
                if nested_arrays:
                    # Debug: show what we have
                    # print(f"Debug: Processing {var_name} with nested_arrays: {list(nested_arrays.keys())}")
                    # for outer_key, inner_dict in nested_arrays.items():
                    #     print(f"  {outer_key}: {list(inner_dict.keys())}")
                    
                    # Sort outer keys numerically
                    sorted_outer_keys = sorted(nested_arrays.keys(), key=lambda x: int(x) if x.isdigit() else 0)
                    
                    sequential_id = 1
                    for outer_key in sorted_outer_keys:
                        inner_dict = nested_arrays[outer_key]
                        
                        # Sort inner keys numerically
                        sorted_inner_keys = sorted(inner_dict.keys(), key=lambda x: int(x) if x.isdigit() else 0)
                        
                        for inner_key in sorted_inner_keys:
                            value = inner_dict[inner_key]
                            lang_parts[var_name]["items"][str(sequential_id)] = value
                            sequential_id += 1
                    
                    # Remove the temporary nested_arrays storage
                    del lang_parts[var_name]["nested_arrays"]
                    
                    total_items = sequential_id - 1
                    print(f"✓ Converted {total_items} nested array items to sequential IDs for {var_name}")
        
        # Add structure metadata
        lang_parts['_metadata'] = {
            'structure_info': structure_info,
            'file_type': structure_info['type'],
            'parsing_pattern': structure_info['pattern']
        }
        
        return lang_parts
        
    except Exception as e:
        print(f"Error reading ActionScript file {as_file_path}: {e}")
        return None

def get_structure_statistics(lang_parts):
    """
    Get statistics about the parsed structure, regardless of the file type.
    """
    if not lang_parts:
        return {}
    
    stats = {
        'total_sections': 0,
        'total_items': 0,
        'sections': {},
        'structure_type': lang_parts.get('_metadata', {}).get('file_type', 'unknown'),
        'parsing_success_rate': 0.0
    }
    
    total_items = 0
    parsed_items = 0
    
    for section_name, section_data in lang_parts.items():
        if section_name.startswith('_'):  # Skip metadata
            continue
            
        if isinstance(section_data, dict) and "items" in section_data:
            items = section_data["items"]
            item_count = len(items)
            
            # Count parsed vs unparsed items
            dict_count = sum(1 for v in items.values() if isinstance(v, dict))
            string_count = sum(1 for v in items.values() if isinstance(v, str))
            
            stats['sections'][section_name] = {
                'total_items': item_count,
                'parsed_items': dict_count,
                'unparsed_items': string_count,
                'success_rate': (dict_count / item_count * 100) if item_count > 0 else 0
            }
            
            total_items += item_count
            parsed_items += dict_count
            stats['total_sections'] += 1
        else:
            stats['sections'][section_name] = {
                'total_items': 0,
                'parsed_items': 0,
                'unparsed_items': 0,
                'success_rate': 100.0
            }
            stats['total_sections'] += 1
    
    stats['total_items'] = total_items
    stats['parsing_success_rate'] = (parsed_items / total_items * 100) if total_items > 0 else 100.0
    
    return stats


In [10]:
# Testing dungeons as file - with inline pattern test
test_line = "DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Salle 1', 'i': 900};"
print(f"Pattern test for: {test_line[:60]}...")
print(f"  '].m[' in line: {'].m[' in test_line}")
print(f"  '] = {{' in line: {'] = {' in test_line}")
print("  line.endswith('}}};'):", test_line.endswith('};'))
print(f"  All match: {'].m[' in test_line and '] = {' in test_line and test_line.endswith('};')}")
print()

dungeons_as_file_path = Path('swf_files/dungeons_pt_1258.as')
if dungeons_as_file_path.exists():
    lang_parts = parse_actionscript_file_generic(dungeons_as_file_path)

    if lang_parts:
        
        # Debugging prints
        print("--- Debugging specific items ---")
        #print(lang_parts)
        if 'DU' in lang_parts and 'items' in lang_parts['DU']:
            items_to_check = ["1", "2"]
            for item_id in items_to_check:
                if item_id in lang_parts['DU']['items']:
                    print(f"Item {item_id}: {lang_parts['DU']['items'][item_id]}")
                else:
                    print(f"Item {item_id} not found.")
        print("------------------------------")
        
        #Dump to JSON for inspection with output_path
        output_json_path = output_path / (dungeons_as_file_path.stem + '.json')
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, ensure_ascii=False, indent=4)

else:
    print(f"ActionScript file not found: {dungeons_as_file_path}")


Pattern test for: DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Salle 1', 'i'...
  '].m[' in line: True
  '] = {' in line: True
  line.endswith('}}};'): True
  All match: True

DEBUG: Dungeon map pattern matched on line 3: DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Sala 1', 'i': 900};
DEBUG: Stored DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Sala 1', 'i': 900}
DEBUG: Dungeon map pattern matched on line 4: DU[1].m[2074] = {'x': 1, 'y': 0, 'z': 0, 'n': 'Sala 2'};
DEBUG: Stored DU[1].m[2074] = {'x': 1, 'y': 0, 'z': 0, 'n': 'Sala 2'}
DEBUG: Dungeon map pattern matched on line 5: DU[1].m[2075] = {'x': 2, 'y': 0, 'z': 0, 'n': 'Sala 3'};
DEBUG: Stored DU[1].m[2075] = {'x': 2, 'y': 0, 'z': 0, 'n': 'Sala 3'}
DEBUG: Dungeon map pattern matched on line 6: DU[1].m[2076] = {'x': 3, 'y': 0, 'z': 0, 'n': 'Sala 4'};
DEBUG: Stored DU[1].m[2076] = {'x': 3, 'y': 0, 'z': 0, 'n': 'Sala 4'}
DEBUG: Dungeon map pattern matched on line 7: DU[1].m[2077] = {'x': 4, 'y': 0, 'z': 0, 'n': 'Sala 5'};
DEBUG: 

In [ ]:
# Quick test for dungeon map pattern
test_line = "DU[1].m[2073] = {'x': 0, 'y': 0, 'z': 0, 'n': 'Salle 1', 'i': 900};"

print(f"Test line: {test_line}")
print(f"'].m[' in line: {'].m[' in test_line}")
print(f"'] = {{' in line: {'] = {' in test_line}")
print(f"line.endswith('}}};'): {test_line.endswith('};')}")
print(f"All conditions: {'].m[' in test_line and '] = {' in test_line and test_line.endswith('};')}")


## Test: LANG File with Raw String Assignments

The `lang_*.as` files contain raw string assignments at the beginning of the file, before any `C = new Object();` declaration. These look like:

```actionscript
A_ASK_MARRIAGE_B = '%1 acceptez-vous d\'épouser %2 ?';
A_ATTACK_B = '<b>%1</b> agresse <b>%2</b>';
ACCEPT = 'Accepter';
```

These are now captured in a `LANG` section with an `items` dictionary using the variable names as keys:

```json
{
  "LANG": {
    "value": "new Object()",
    "items": {
      "A_ASK_MARRIAGE_B": "%1 acceptez-vous d'épouser %2 ?",
      "A_ATTACK_B": "<b>%1</b> agresse <b>%2</b>",
      "ACCEPT": "Accepter"
    }
  }
}
```

In [8]:
# Test parsing of lang_fr_1254.as file with raw string assignments
print("🧪 Testing LANG file with raw string assignments...")
lang_as_file_path = Path('swf_files/lang_pt_1254.as')

if lang_as_file_path.exists():
    print(f"Parsing LANG file: {lang_as_file_path}")
    
    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(lang_as_file_path)
    
    if lang_parts:
        # Check if LANG section exists with items
        if 'LANG' in lang_parts and 'items' in lang_parts['LANG']:
            lang_items = lang_parts['LANG']['items']
            print(f"✅ Found LANG section with {len(lang_items)} raw string items")
            
            # Show first 10 items as sample
            print("\nSample items from LANG section:")
            for i, (key, value) in enumerate(list(lang_items.items())[:10], 1):
                # Truncate long values for display
                display_value = value[:60] + '...' if len(value) > 60 else value
                print(f"  {i}. {key} = '{display_value}'")
            
            if len(lang_items) > 10:
                print(f"  ... and {len(lang_items) - 10} more items")
            
            # Check for expected keys
            expected_keys = ['A_ASK_MARRIAGE_B', 'A_ATTACK_B', 'ACCEPT', 'ACCESS_DENIED']
            
            print("\nVerifying expected keys:")
            for key in expected_keys:
                if key in lang_items:
                    value = lang_items[key][:50] + '...' if len(lang_items[key]) > 50 else lang_items[key]
                    print(f"  ✅ {key}: '{value}'")
                else:
                    print(f"  ❌ {key}: NOT FOUND")
        else:
            print("❌ LANG section not found in parsed output")
        
        # Check other sections too
        other_sections = [k for k in lang_parts.keys() if k != 'LANG' and not k.startswith('_')]
        if other_sections:
            print(f"\n📋 Other sections found: {', '.join(other_sections[:10])}")
            if len(other_sections) > 10:
                print(f"    ... and {len(other_sections) - 10} more")
        
        # Save to JSON for inspection
        output_json_path = output_path / (lang_as_file_path.stem + '.json')
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, ensure_ascii=False, indent=2)
        print(f"\n💾 Saved parsed output to: {output_json_path}")
    else:
        print("❌ Failed to parse file")
else:
    print(f"❌ ActionScript file not found: {lang_as_file_path}")

🧪 Testing LANG file with raw string assignments...
Parsing LANG file: swf_files\lang_pt_1254.as
✅ Found LANG section with 2635 raw string items

Sample items from LANG section:
  1. A_ASK_MARRIAGE_B = '%1, você deseja pedir a mão de %2 em casamento?'
  2. A_ATTACK_B = '<b>%1</b> agride <b>%2</b>'
  3. ACCEPT = 'Aceitar'
  4. ACCESS_DENIED = 'Login ou senha incorretos.'
  5. ACCESS_DENIED_AUTHENTICATOR = 'Esta conta está protegida pelo Authenticator\n1. Acesse o ap...'
  6. ACCESS_DENIED_MINICLIP = 'Acesso negado. ID ou senha inválidos.'
  7. ACCOUNT = 'Credencial'
  8. ACCOUNT_INFO = 'Detalhes da conta'
  9. A_CHALENGE_B = '<b>%1</b> desafia <b>%2</b>'
  10. A_CHALENGE_YOU = '<b>%1</b> te desafia. Você aceita?'
  ... and 2625 more items

Verifying expected keys:
  ✅ A_ASK_MARRIAGE_B: '%1, você deseja pedir a mão de %2 em casamento?'
  ✅ A_ATTACK_B: '<b>%1</b> agride <b>%2</b>'
  ✅ ACCEPT: 'Aceitar'
  ✅ ACCESS_DENIED: 'Login ou senha incorretos.'

📋 Other sections found: C, C.CHAT_FILTE

In [9]:
print("🧪 Testing improved GENERIC ActionScript parsing...")
kb_as_file_path = Path("swf_files/kb_pt_1248.as")

if kb_as_file_path.exists():
    print(f"Parsing KB file: {kb_as_file_path}")

    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(kb_as_file_path)

    if lang_parts:
        
        # Step 3: Save JSON to output directory
        json_filename = kb_as_file_path.stem + '.json'
        json_output_path = output_path / json_filename
        
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, indent=2, ensure_ascii=False)
    else:
        print("❌ Failed to parse file")

🧪 Testing improved GENERIC ActionScript parsing...
Parsing KB file: swf_files\kb_pt_1248.as


In [ ]:
# Test the generic implementation

print("🧪 Testing improved GENERIC ActionScript parsing...")
kb_as_file_path = Path("swf_files/kb_pt_1248.as")

if kb_as_file_path.exists():
    print(f"Parsing KB file: {kb_as_file_path}")

    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(kb_as_file_path)

    if lang_parts:
        
        # Step 3: Save JSON to output directory
        json_filename = kb_as_file_path.stem + '.json'
        json_output_path = output_path / json_filename
        
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, indent=2, ensure_ascii=False)
    else:
        print("❌ Failed to parse file")

# Test: timezones_fr_1248.as - Month names array parsing
timezones_as_file_path = Path('swf_files/timezones_fr_1248.as')
parsed = parse_actionscript_file_generic(timezones_as_file_path)

# Expected 12 month names in order
expected_months = {
    '0': 'Javian',
    '1': 'Flovor',
    '2': 'Martalo',
    '3': 'Aperirel',
    '4': 'Maisial',
    '5': 'Juinssidor',
    '6': 'Jouillier',
    '7': 'Fraouctor',
    '8': 'Septange',
    '9': 'Octolliard',
    '10': 'Novamaire',
    '11': 'Descendre'
}

#Dump to JSON for inspection with output_path
if parsed:
    output_json_path = output_path / (timezones_as_file_path.stem + '.json')
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(parsed, f, ensure_ascii=False, indent=4)

print(parsed)
if 'T.m' in parsed and 'items' in parsed['T.m']:
    month_items = parsed['T.m']['items']
    print(f"Found {len(month_items)} month items")
    
    all_correct = True
    for month_id, expected_name in expected_months.items():
        if month_id in month_items:
            actual_name = month_items[month_id]
            if actual_name == expected_name:
                print(f"✓ Month {month_id}: {actual_name}")
            else:
                print(f"✗ Month {month_id}: Expected '{expected_name}', got '{actual_name}'")
                all_correct = False
        else:
            print(f"✗ Month {month_id}: Missing! (expected '{expected_name}')")
            all_correct = False
    
    if all_correct:
        print("\n✅ All 12 months parsed correctly!")
    else:
        print("\n❌ Some months incorrect or missing")
else:
    print("❌ Section 'T' not found or has no items")
    print(f"Available sections: {list(parsed.keys())}")


# Test the generic parser on the spells file
spells_as_file_path = Path('swf_files/spells_fr_1254.as')
if spells_as_file_path.exists():
    lang_parts = parse_actionscript_file_generic(spells_as_file_path)

    if lang_parts:
        
        # Debugging prints
        print("--- Debugging specific items ---")
        #print(lang_parts)
        if 'S' in lang_parts and 'items' in lang_parts['S']:
            items_to_check = ["688", "691", "694", "4118"]
            for item_id in items_to_check:
                if item_id in lang_parts['S']['items']:
                    print(f"Item {item_id}: {lang_parts['S']['items'][item_id]}")
                else:
                    print(f"Item {item_id} not found.")
        print("------------------------------")
        
        #Dump to JSON for inspection with output_path
        output_json_path = output_path / (spells_as_file_path.stem + '.json')
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, ensure_ascii=False, indent=4)

else:
    print(f"ActionScript file not found: {spells_as_file_path}")

# Test nested array parsing with subtitles file
subtitles_as_file_path = Path('swf_files/subtitles_fr_1254.as')
if subtitles_as_file_path.exists():
    print(f"🧪 Testing nested array parsing...")
    print(f"Parsing SUBTITLES file: {subtitles_as_file_path}\n")
    
    lang_parts = parse_actionscript_file_generic(subtitles_as_file_path)

    if lang_parts:
        # Expected order from user's specification (using raw strings to preserve literal \n)
        expected_items = {
            "1": r"Dans ce Monde, il existe 6 talismans\nporteurs de puissance et de prospérité.",
            "2": "Six oeufs de Dragon nommés Dofus",
            "3": "Qui furent dérobés à leurs gardiens...",
            "4": r"Alors, les éléments se déchaînèrent,\nLes monstres se brossèrent les crocs,",
            "5": r"Les aventuriers brandirent leurs lames,\nEt ce fut un joyeux carnage !",
            "6": r"Mais le temps est venu !\nTa destinée te mène à Incarnam...",
            "7": "Deviens chevalier, guérisseur, invocateur,",
            "8": r"Retrouve les Dofus,\nEt deviens le héros que le Monde attend !",
            "9": "[TR2]",
            "10": "[TR2]",
            "11": "[TR2]",
            "12": "[TR2]",
            "13": "[TR2]"
        }
        
        # Check if SUB section exists
        if 'SUB' in lang_parts and 'items' in lang_parts['SUB']:
            sub_items = lang_parts['SUB']['items']
            print(f"✅ SUB section found with {len(sub_items)} items\n")
            
            # Verify all expected items
            print("Verifying sequential ordering:")
            print("  ID | Expected (first 50 chars)              | Got (first 50 chars)                   | Status")
            print("  " + "-" * 100)
            
            all_correct = True
            for item_id in sorted(expected_items.keys(), key=lambda x: int(x)):
                expected_value = expected_items[item_id]
                actual_value = sub_items.get(item_id, "NOT FOUND")
                
                # Compare (both are already in the format with actual newlines, not escaped)
                # The expected_items dict should use actual \n not escaped \\n
                is_correct = actual_value == expected_value
                all_correct = all_correct and is_correct
                
                status = "✅" if is_correct else "❌"
                expected_preview = expected_value[:50].replace('\n', '\\n')
                actual_preview = str(actual_value)[:50].replace('\n', '\\n')
                
                print(f"  {item_id:>2} | {expected_preview:40} | {actual_preview:40} | {status}")
            
            if all_correct:
                print("\n🎉 All items match! Nested array parsing working correctly.")
            else:
                print(f"\n⚠️ Some items don't match.")
        else:
            print("❌ SUB section not found in parsed output")
        
        # Dump to JSON for inspection
        output_json_path = output_path / (subtitles_as_file_path.stem + '.json')
        print(f"\nSaving parsed output to: {output_json_path}")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, ensure_ascii=False, indent=4)
        
        print(f"\n💾 Saved parsed output to: {output_json_path}")
else:
    print(f"❌ ActionScript file not found: {subtitles_as_file_path}")

# Test SRVC parsing with pipe-separated keys - WITH CORRECT EXPECTED VALUES
print("🧪 Testing SRVC pipe-key parsing...")
servers_as_file_path = Path("swf_files/servers_fr_1254.as")

if servers_as_file_path.exists():
    print(f"Parsing SERVERS file: {servers_as_file_path}\n")

    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(servers_as_file_path)

    if lang_parts:
        stats = get_structure_statistics(lang_parts)
        
        print(f"✅ Successfully parsed! File type: {stats['structure_type']}")
        print(f"📊 Statistics:")
        print(f"   Total sections: {stats['total_sections']}")
        print(f"   Total items: {stats['total_items']}")
        print(f"   Parsing success rate: {stats['parsing_success_rate']:.1f}%\n")
        
        # Check SRVC section specifically
        if 'SRVC' in lang_parts and 'items' in lang_parts['SRVC']:
            srvc_items = lang_parts['SRVC']['items']
            print(f"✅ SRVC section found with {len(srvc_items)} items\n")
            
            # CORRECT expected values from your reference data
            expected_items= {
                1: "Clan de Sériane-Kerm",      # SRVC.1|1
                2: "Clan des Samoulailles",     # SRVC.1|2
                3: "Kawet",                     # SRVC.2|1
                4: "(Ne pas traduire)",         # SRVC.2|2
                5: "Sérianiseur",               # SRVC.3|1
                6: "Samoulailliseur",           # SRVC.3|2
                7: "Sériane",                   # SRVC.4|1
                8: "Samoulaille",               # SRVC.4|2
                9: "SHADOW DOOM HELL§§§",       # SRVC.1|666
                10: "de Nimaoh",                # SRVC.4|666
                11: "Etage Zooooooornal",       # SRVC.31|3
                12: "La Caravane des Nephthys", # SRVC.1|4
                13: "Nephthysateur",            # SRVC.3|4
                14: "Nephthys",                 # SRVC.4|4
                15: "Clan des Stères",          # SRVC.1|3
                16: "Stèrisateur",              # SRVC.3|3
                17: "Stère",                    # SRVC.4|3
                18: "Clan des Selenytes",       # SRVC.1|5
                19: "Selenyte",                 # SRVC.4|5
                20: "Selenisateur",             # SRVC.3|5
                21: "Clan de Nédora Riem",      # SRVC.1|6
                22: "Nédorisateur",             # SRVC.3|6
                23: "Nédora",                   # SRVC.4|6
            }
            
            print("Verifying all expected items:")
            print("  ID | Expected                   | Got                        | Status")
            print("  " + "-"*80)
            
            all_correct = True
            for item_id in sorted([int(k) for k in expected_items.keys()]):
                item_id_str = str(item_id)
                expected_value = expected_items[item_id]
                actual_value = srvc_items.get(item_id_str, "NOT FOUND")
                is_correct = actual_value == expected_value
                status = "✅" if is_correct else "❌"
                print(f"  {item_id:2} | {expected_value:26} | {actual_value:26} | {status}")
                if not is_correct:
                    all_correct = False
            
            if all_correct:
                print("\n🎉 All items match! SRVC parsing working correctly.")
            else:
                print(f"\n⚠️  {sum(1 for k in expected_items.keys() if srvc_items.get(k) != expected_items[k])} items don't match expected values")
        
        # Save the parsed output for inspection
        json_filename = servers_as_file_path.stem + '.json'
        json_output_path = output_path / json_filename
        
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Saved parsed output to: {json_output_path}")
    else:
        print("❌ Failed to parse file")
else:
    print(f"❌ Servers ActionScript file not found: {servers_as_file_path}")

# Test the generic parser on the items file
items_as_file_path = Path('swf_files/items_fr_1260.as')
if items_as_file_path.exists():
    lang_parts = parse_actionscript_file_generic(items_as_file_path)
    
    if lang_parts:
        stats = get_structure_statistics(lang_parts)
        unparsed_count = stats.get('sections', {}).get('I.u', {}).get('unparsed_items', 0)
        print(f"✅ Successfully parsed! File type: {stats['structure_type']}")
        print(f"📊 Statistics:")
        print(f"   Total sections: {stats['total_sections']}")
        print(f"   Total items: {stats['total_items']}")
        print(f"   Parsing success rate: {stats['parsing_success_rate']:.1f}%")
        
        for section_name, section_stats in stats['sections'].items():
            if section_stats['total_items'] > 0:
                print(f"   - {section_name}: {section_stats['total_items']} items ({section_stats['success_rate']:.1f}% parsed)")
        
        if unparsed_count == 0:
            print("✅ All items parsed successfully!")
        else:
            print(f"⚠️ Found {unparsed_count} unparsed items.")

        # Debugging prints
        print("--- Debugging specific items ---")
        if 'I.u' in lang_parts and 'items' in lang_parts['I.u']:
            items_to_check = ["251", "378", "401", "674"]
            for item_id in items_to_check:
                if item_id in lang_parts['I.u']['items']:
                    print(f"Item {item_id}: {lang_parts['I.u']['items'][item_id]}")
                else:
                    print(f"Item {item_id} not found.")
        print("------------------------------")
        
        #Dump to JSON for inspection
        output_json_path = output_path / 'items_fr_1260.json'
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, ensure_ascii=False, indent=4)

else:
    print(f"Items ActionScript file not found: {items_as_file_path}")

print("🧪 Testing improved GENERIC ActionScript parsing...")
hint_as_file_path = Path("swf_files/hints_fr_1254.as")

if hint_as_file_path.exists():
    print(f"Parsing HINT file: {hint_as_file_path}")

    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(hint_as_file_path)

    if lang_parts:
        
        # Step 3: Save JSON to output directory
        json_filename = hint_as_file_path.stem + '.json'
        json_output_path = output_path / json_filename
        
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, indent=2, ensure_ascii=False)
    else:
        print("❌ Failed to parse file")

print("🧪 Testing improved GENERIC ActionScript parsing...")
dungeon_as_file_path = Path("swf_files/dungeons_pt_1258.as")

if dungeon_as_file_path.exists():
    print(f"Parsing DUNGEON file: {dungeon_as_file_path}")

    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(dungeon_as_file_path)

    if lang_parts:
        stats = get_structure_statistics(lang_parts)
        
        print(f"✅ Successfully parsed! File type: {stats['structure_type']}")
        print(f"📊 Statistics:")
        print(f"   Total sections: {stats['total_sections']}")
        print(f"   Total items: {stats['total_items']}")
        print(f"   Parsing success rate: {stats['parsing_success_rate']:.1f}%")
        
        for section_name, section_stats in stats['sections'].items():
            if section_stats['total_items'] > 0:
                print(f"   - {section_name}: {section_stats['total_items']} items ({section_stats['success_rate']:.1f}% parsed)")
        # dump parsed data for inspection
        # Step 3: Save JSON to output directory
        json_filename = dungeon_as_file_path.stem + '.json'
        json_output_path = output_path / json_filename
        
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(lang_parts, f, indent=2, ensure_ascii=False)
    else:
        print("❌ Failed to parse file")


print("🧪 Testing improved GENERIC ActionScript parsing...")
quest_as_file_path = Path("swf_files/quests_fr_1248.as")

if quest_as_file_path.exists():
    print(f"Parsing QUESTS file: {quest_as_file_path}")

    # Parse the ActionScript file
    lang_parts = parse_actionscript_file_generic(quest_as_file_path)

    if lang_parts:
        stats = get_structure_statistics(lang_parts)
        
        print(f"✅ Successfully parsed! File type: {stats['structure_type']}")
        print(f"📊 Statistics:")
        print(f"   Total sections: {stats['total_sections']}")
        print(f"   Total items: {stats['total_items']}")
        print(f"   Parsing success rate: {stats['parsing_success_rate']:.1f}%")
        
        for section_name, section_stats in stats['sections'].items():
            if section_stats['total_items'] > 0:
                print(f"   - {section_name}: {section_stats['total_items']} items ({section_stats['success_rate']:.1f}% parsed)")

    else:
        print("❌ Failed to parse file")

# Test with an NPC file
print(f"\n🧪 Testing with NPC file...")
npc_file_path = Path("swf_files/npc_fr_1254.as")

if npc_file_path.exists():
    print(f"Parsing NPC file: {npc_file_path}")
    
    npc_parts = parse_actionscript_file_generic(npc_file_path)
    
    if npc_parts:
        npc_stats = get_structure_statistics(npc_parts)
        
        print(f"✅ Successfully parsed! File type: {npc_stats['structure_type']}")
        print(f"📊 Statistics:")
        print(f"   Total sections: {npc_stats['total_sections']}")
        print(f"   Total items: {npc_stats['total_items']}")
        print(f"   Parsing success rate: {npc_stats['parsing_success_rate']:.1f}%")
        
        for section_name, section_stats in npc_stats['sections'].items():
            if section_stats['total_items'] > 0:
                print(f"   - {section_name}: {section_stats['total_items']} items ({section_stats['success_rate']:.1f}% parsed)")
    else:
        print("❌ Failed to parse NPC file")

# Test with a Spells file
print(f"\n🧪 Testing with SPELLS file...")
spells_file_path = Path("swf_files/spells_fr_1254.as")

if spells_file_path.exists():
    print(f"Parsing SPELLS file: {spells_file_path}")
    
    spells_parts = parse_actionscript_file_generic(spells_file_path)
    
    if spells_parts:
        spells_stats = get_structure_statistics(spells_parts)
        
        print(f"✅ Successfully parsed! File type: {spells_stats['structure_type']}")
        print(f"📊 Statistics:")
        print(f"   Total sections: {spells_stats['total_sections']}")
        print(f"   Total items: {spells_stats['total_items']}")
        print(f"   Parsing success rate: {spells_stats['parsing_success_rate']:.1f}%")
        
        for section_name, section_stats in spells_stats['sections'].items():
            if section_stats['total_items'] > 0:
                print(f"   - {section_name}: {section_stats['total_items']} items ({section_stats['success_rate']:.1f}% parsed)")
    else:
        print("❌ Failed to parse SPELLS file")

print(f"\n🎉 Generic parser can now handle different ActionScript structures!")

🧪 Testing SRVC pipe-key parsing...
Parsing SERVERS file: swf_files\servers_fr_1254.as

✓ Converted 23 SRVC pipe-keyed items to sequential IDs for SRVC
✅ Successfully parsed! File type: multi_root
📊 Statistics:
   Total sections: 6
   Total items: 55
   Parsing success rate: 36.4%

✅ SRVC section found with 23 items

Verifying all expected items:
  ID | Expected                   | Got                        | Status
  --------------------------------------------------------------------------------
   1 | Clan de Sériane-Kerm       | Clan de Sériane-Kerm       | ✅
   2 | Clan des Samoulailles      | Clan des Samoulailles      | ✅
   3 | Kawet                      | Kawet                      | ✅
   4 | (Ne pas traduire)          | (Ne pas traduire)          | ✅
   5 | Sérianiseur                | Sérianiseur                | ✅
   6 | Samoulailliseur            | Samoulailliseur            | ✅
   7 | Sériane                    | Sériane                    | ✅
   8 | Samoulaille          

In [94]:
# Test: dungeons_fr_1258.as - Dungeon maps parsing

parsed = parse_actionscript_file_generic(dungeon_as_file_path)

# Check if dungeons were parsed
if 'DU' in parsed and 'items' in parsed['DU']:
    dungeons = parsed['DU']['items']
    print(f"Found {len(dungeons)} dungeons")
    
    # Check specific dungeons with maps
    test_dungeons = {
        '1': {'name': 'Donjon Bouftou', 'map_count': 11, 'sample_map': '2073'},
        '2': {'name': 'Donjon des champs', 'map_count': 8, 'sample_map': '9771'}
    }
    
    for dungeon_id, expected in test_dungeons.items():
        if dungeon_id in dungeons:
            dungeon = dungeons[dungeon_id]
            
            # Check if it's a dict with 'n' and 'm' keys
            if isinstance(dungeon, dict):
                dungeon_name = dungeon.get('n', 'MISSING')
                maps = dungeon.get('m', {})
                
                print(f"\n✓ Dungeon {dungeon_id}: {dungeon_name}")
                print(f"  Expected name: {expected['name']}")
                print(f"  Maps found: {len(maps)}")
                print(f"  Expected maps: {expected['map_count']}")
                
                # Check sample map
                if expected['sample_map'] in maps:
                    sample_map = maps[expected['sample_map']]
                    print(f"  ✓ Sample map {expected['sample_map']}: {sample_map}")
                else:
                    print(f"  ✗ Sample map {expected['sample_map']} not found")
                    print(f"  Available maps: {list(maps.keys())[:5]}...")
            else:
                print(f"\n✗ Dungeon {dungeon_id}: Not a dict, got {type(dungeon)}")
        else:
            print(f"\n✗ Dungeon {dungeon_id}: Not found")
    
    # Show structure of first dungeon
    if '1' in dungeons and isinstance(dungeons['1'], dict):
        print(f"\n📋 Structure of DU.1:")
        print(f"  Keys: {list(dungeons['1'].keys())}")
        if 'm' in dungeons['1']:
            print(f"  Maps IDs: {list(dungeons['1']['m'].keys())}")
            first_map_id = list(dungeons['1']['m'].keys())[0] if dungeons['1']['m'] else None
            if first_map_id:
                print(f"  First map structure: {dungeons['1']['m'][first_map_id]}")
else:
    print("❌ Section 'DU' not found or has no items")
    print(f"Available sections: {list(parsed.keys())}")


Found 46 dungeons

✓ Dungeon 1: Calabouço Gobball
  Expected name: Donjon Bouftou
  Maps found: 0
  Expected maps: 11
  ✗ Sample map 2073 not found
  Available maps: []...

✓ Dungeon 2: Calabouço das Margaridas
  Expected name: Donjon des champs
  Maps found: 0
  Expected maps: 8
  ✗ Sample map 9771 not found
  Available maps: []...

📋 Structure of DU.1:
  Keys: ['n', 'm']
  Maps IDs: []


In [88]:
# Process all SWF files in the swf_files directory using the GENERIC parser
flare_path = get_flare_path()
swf_files = list(swf_path.glob("*.swf"))

if not swf_files:
    print("No SWF files found in swf_files directory.")
else:
    print(f"Found {len(swf_files)} SWF files to process\n")
    
    # Track processing statistics
    processing_stats = {
        'total_files': len(swf_files),
        'swf_extraction_failed': [],
        'json_parsing_failed': [],
        'successful_files': [],
        'file_type_stats': {},  # Track stats by file type
        'language_data': {}
    }
    
    parsed_data = {}
    
    for swf_file in swf_files:
        file_lang = swf_file.stem  # e.g., "quests_en_1248"
        print(f"Processing {swf_file.name}...", end=" ")
        
        # Step 1: Extract SWF to ActionScript
        as_file = extract_swf_to_actionscript(swf_file, flare_path)
        
        if as_file:
            # Step 2: Parse ActionScript to JSON using GENERIC parser
            json_data = parse_actionscript_file_generic(as_file)
            
            if json_data:  # Check if parsing was successful
                # Step 3: Save JSON to output directory
                json_filename = swf_file.stem + '.json'
                json_output_path = output_path / json_filename
                
                with open(json_output_path, 'w', encoding='utf-8') as f:
                    json.dump(json_data, f, indent=2, ensure_ascii=False)
                
                parsed_data[swf_file.stem] = json_data
                processing_stats['successful_files'].append(file_lang)
                
                # Extract GENERIC statistics using the new function
                try:
                    stats = get_structure_statistics(json_data)
                    file_type = stats.get('structure_type', 'unknown')
                
                    # Track by file type
                    if file_type not in processing_stats['file_type_stats']:
                        processing_stats['file_type_stats'][file_type] = {
                            'files': [],
                            'total_items': 0,
                            'total_sections': 0,
                            'avg_success_rate': 0.0
                        }
                    
                    type_stats = processing_stats['file_type_stats'][file_type]
                    type_stats['files'].append(file_lang)
                    type_stats['total_items'] += stats['total_items']
                    type_stats['total_sections'] += stats['total_sections']
                    
                    # Store detailed language statistics  
                    processing_stats['language_data'][file_lang] = {
                        'file_type': file_type,
                        'sections': list(stats['sections'].keys()),
                        'total_items': stats['total_items'],
                        'parsing_success_rate': stats['parsing_success_rate'],
                        'section_details': stats['sections']
                    }
                    
                    print("✓")

                except Exception as e:
                    processing_stats['json_parsing_failed'].append(file_lang)
                    print(f"✗ JSON parsing failed: {e}")
            else:
                processing_stats['json_parsing_failed'].append(file_lang)
                print(f"✗ JSON parsing failed: {e}")
        else:
            processing_stats['swf_extraction_failed'].append(file_lang)
            print("✗ SWF extraction failed")
    
    # ===== IMPROVED SUMMARY REPORTING =====
    print(f"\n{'='*80}")
    print("GENERIC ACTIONSCRIPT PROCESSING SUMMARY")
    print(f"{'='*80}")
    
    # Overall success rates
    total = processing_stats['total_files']
    extraction_failures = len(processing_stats['swf_extraction_failed'])
    parsing_failures = len(processing_stats['json_parsing_failed'])
    successes = len(processing_stats['successful_files'])
    
    extraction_success_rate = ((total - extraction_failures) / total) * 100
    parsing_success_rate = ((total - extraction_failures - parsing_failures) / (total - extraction_failures)) * 100 if (total - extraction_failures) > 0 else 0
    overall_success_rate = (successes / total) * 100
    
    print(f"📊 OVERALL STATISTICS:")
    print(f"  Total files: {total}")
    print(f"  SWF extraction: {total - extraction_failures}/{total} ({extraction_success_rate:.1f}% success)")
    print(f"  JSON parsing: {total - extraction_failures - parsing_failures}/{total - extraction_failures} ({parsing_success_rate:.1f}% success)")
    print(f"  Overall success: {successes}/{total} ({overall_success_rate:.1f}%)")
    
    # Failed files details
    if extraction_failures > 0:
        print(f"\n❌ SWF EXTRACTION FAILURES ({extraction_failures}):")
        for failed_file in processing_stats['swf_extraction_failed']:
            print(f"    - {failed_file}")
    
    if parsing_failures > 0:
        print(f"\n❌ JSON PARSING FAILURES ({parsing_failures}):")
        for failed_file in processing_stats['json_parsing_failed']:
            print(f"    - {failed_file}")
    
    # File type breakdown
    if processing_stats['file_type_stats']:
        print(f"\n🗂️ FILE TYPE BREAKDOWN:")
        for file_type, type_data in processing_stats['file_type_stats'].items():
            file_count = len(type_data['files'])
            total_items = type_data['total_items']
            total_sections = type_data['total_sections']
            
            # Calculate average success rate for this file type
            success_rates = []
            for file_name in type_data['files']:
                if file_name in processing_stats['language_data']:
                    success_rates.append(processing_stats['language_data'][file_name]['parsing_success_rate'])
            avg_success_rate = sum(success_rates) / len(success_rates) if success_rates else 0
            
            print(f"  📁 {file_type.upper()}: {file_count} files")
            print(f"     Total items: {total_items:,}")
            print(f"     Total sections: {total_sections}")
            print(f"     Avg success rate: {avg_success_rate:.1f}%")
            
            # Show sample files
            sample_files = type_data['files'][:3]
            if len(type_data['files']) > 3:
                sample_files_str = ", ".join(sample_files) + f", ... (+{len(type_data['files'])-3} more)"
            else:
                sample_files_str = ", ".join(sample_files)
            print(f"     Files: {sample_files_str}")
    
    # Multi-language analysis
    if len(processing_stats['language_data']) > 1:
        print(f"\n🌍 MULTI-LANGUAGE ANALYSIS:")
        
        # Extract language codes and base filenames
        lang_analysis = {}
        for file_lang in processing_stats['language_data'].keys():
            # Parse filename like "quests_fr_1248" -> base: "quests", lang: "fr"
            parts = file_lang.split('_')
            if len(parts) >= 2:
                base_name = parts[0]
                lang_code = parts[1]
                
                if base_name not in lang_analysis:
                    lang_analysis[base_name] = {}
                
                file_data = processing_stats['language_data'][file_lang]
                lang_analysis[base_name][lang_code] = {
                    'items': file_data['total_items'],
                    'success_rate': file_data['parsing_success_rate'],
                    'file_type': file_data['file_type']
                }
        
        # Display analysis by base name (e.g., quests, npc, spells)
        for base_name, languages in lang_analysis.items():
            print(f"  📋 {base_name.upper()}:")
            langs = sorted(languages.keys())
            for lang in langs:
                data = languages[lang]
                print(f"     {lang}: {data['items']:,} items ({data['success_rate']:.1f}% parsed) - {data['file_type']}")
            
            # Show item count consistency
            item_counts = [languages[lang]['items'] for lang in langs]
            if len(set(item_counts)) == 1:
                print(f"     ✅ Consistent item counts across languages")
            else:
                min_count = min(item_counts)
                max_count = max(item_counts)
                print(f"     ⚠️ Item count varies: {min_count:,} to {max_count:,}")
    
    print(f"\n✅ Processing complete! {successes} files successfully processed.")
    print(f"📁 All parsed files saved to: {output_path}")
    
    # Show specific examples for problematic cases verification
    if successes > 0:
        print(f"\n🔍 VERIFICATION: Check problematic cases are fixed...")
        
        # Check if we have quest files with the problematic items
        quest_files = [f for f in processing_stats['successful_files'] if f.startswith('quests_')]
        if quest_files:
            sample_quest_file = quest_files[0]
            if sample_quest_file in parsed_data:
                quest_data = parsed_data[sample_quest_file]
                if "Q.s" in quest_data and "items" in quest_data["Q.s"]:
                    items = quest_data["Q.s"]["items"]
                    
                    # Check for the specific problematic items
                    test_items = ["3", "189"]
                    for item_id in test_items:
                        if item_id in items:
                            item_data = items[item_id]
                            if isinstance(item_data, dict):
                                print(f"     ✅ Item '{item_id}' properly parsed: {item_data.get('n', 'No name')}")
                            else:
                                print(f"     ❌ Item '{item_id}' still unparsed: {str(item_data)[:50]}...")
        
        # Check NPC files for apostrophe handling
        npc_files = [f for f in processing_stats['successful_files'] if f.startswith('npc_')]
        if npc_files:
            sample_npc_file = npc_files[0]
            if sample_npc_file in parsed_data:
                npc_data = parsed_data[sample_npc_file]
                if "N.d" in npc_data and "items" in npc_data["N.d"]:
                    npc_items = npc_data["N.d"]["items"]
                    
                    # Find an NPC with apostrophes
                    for npc_id, npc_data_item in list(npc_items.items())[:10]:
                        if isinstance(npc_data_item, dict) and 'n' in npc_data_item:
                            name = npc_data_item['n']
                            if "'" in name:
                                print(f"     ✅ NPC '{npc_id}' apostrophe handled: {name}")
                                break

Found 152 SWF files to process

Processing alignment_en_1254.swf... Successfully extracted alignment_en_1254.swf -> alignment_en_1254.as
✓
Processing alignment_es_1254.swf... Successfully extracted alignment_es_1254.swf -> alignment_es_1254.as
✓
Processing alignment_fr_1254.swf... Successfully extracted alignment_fr_1254.swf -> alignment_fr_1254.as
✓
Processing alignment_pt_1254.swf... Successfully extracted alignment_pt_1254.swf -> alignment_pt_1254.as
✓
Processing audio_en_1248.swf... Successfully extracted audio_en_1248.swf -> audio_en_1248.as
✓
Processing audio_es_1248.swf... Successfully extracted audio_es_1248.swf -> audio_es_1248.as
✓
Processing audio_en_1248.swf... Successfully extracted audio_en_1248.swf -> audio_en_1248.as
✓
Processing audio_es_1248.swf... Successfully extracted audio_es_1248.swf -> audio_es_1248.as
✓
Processing audio_fr_1248.swf... Successfully extracted audio_fr_1248.swf -> audio_fr_1248.as
✓
Processing audio_pt_1248.swf... Successfully extracted audio_pt_1

# Merged Quest relationships file

In [ ]:
import json
from pathlib import Path

def extract_and_merge_quest_relationships():
    """
    Extract quest-to-step and step-to-objective relationships from custom config
    and merge them into the parsed ActionScript data (IDs and metadata only, no localization)
    
    Removes all string values that will be loaded later by localization files:
    - Quest/Step/Objective names ("n")
    - Quest/Step/Objective descriptions ("d" when string)
    - Objective parameters containing strings ("p" array with strings)
    - Quest objective type templates ("Q.t" strings like "Aller voir #1" → null)
    """
    
    # Load the parsed ActionScript data
    parsed_file = Path("output/quests_fr_1248.json")
    custom_file = Path("api/custom/quests.json")
    
    if not parsed_file.exists():
        print(f"❌ Parsed file not found: {parsed_file}")
        return
    
    if not custom_file.exists():
        print(f"❌ Custom file not found: {custom_file}")
        return
    
    # Load data
    with open(parsed_file, 'r', encoding='utf-8') as f:
        parsed_data = json.load(f)
    
    with open(custom_file, 'r', encoding='utf-8-sig') as f:
        custom_data = json.load(f)
    
    def remove_localization_strings(data_dict):
        """Remove localization strings from any data structure"""
        if not isinstance(data_dict, dict):
            return data_dict
            
        cleaned = {}
        for key, value in data_dict.items():
            if key in ['n', 'd'] and isinstance(value, str):
                # Skip localized names and string descriptions
                continue
            elif key == 'p' and isinstance(value, list):
                # Filter out string parameters, keep only non-string values
                filtered_params = [p for p in value if not isinstance(p, str)]
                if filtered_params:
                    cleaned[key] = filtered_params
            elif isinstance(value, dict):
                cleaned_value = remove_localization_strings(value)
                if cleaned_value:  # Only add if not empty
                    cleaned[key] = cleaned_value
            elif isinstance(value, list):
                # Keep lists but filter string elements in parameter arrays
                if key == 'p':
                    filtered_list = [item for item in value if not isinstance(item, str)]
                    if filtered_list:
                        cleaned[key] = filtered_list
                else:
                    cleaned[key] = value
            else:
                # Keep all non-string values (IDs, numbers, booleans)
                cleaned[key] = value
                
        return cleaned
    
    print("🔍 Analyzing current data structure...")
    
    # Count current relationships
    quests_with_steps = 0
    steps_with_objectives = 0
    
    if 'Q.q' in parsed_data and 'items' in parsed_data['Q.q']:
        for quest_id, quest_data in parsed_data['Q.q']['items'].items():
            if isinstance(quest_data, dict) and 's' in quest_data and quest_data['s']:
                quests_with_steps += 1
    
    if 'Q.s' in parsed_data and 'items' in parsed_data['Q.s']:
        for step_id, step_data in parsed_data['Q.s']['items'].items():
            if isinstance(step_data, dict) and 'o' in step_data and step_data['o']:
                steps_with_objectives += 1
    
    print(f"📊 Current state:")
    print(f"   Quests with steps: {quests_with_steps}")
    print(f"   Steps with objectives: {steps_with_objectives}")
    
    # Extract quest-to-step relationships from custom data
    quest_step_mappings = {}
    if 'CQ.q' in custom_data:
        for quest_custom in custom_data['CQ.q']:
            quest_id = str(quest_custom.get('id'))
            if 'steps' in quest_custom:  # If custom data already has steps
                quest_step_mappings[quest_id] = quest_custom['steps']
    
    # Extract step-to-objective relationships from custom data
    step_objective_mappings = {}
    step_metadata = {}
    
    if 'CQ.s' in custom_data:
        for step_custom in custom_data['CQ.s']:
            step_id = str(step_custom.get('id'))
            if step_id:
                # Extract objective mappings (IDs only)
                objectives = step_custom.get('o', [])
                if objectives:
                    step_objective_mappings[step_id] = objectives
                
                # Extract non-localized metadata
                metadata = {}
                if 'd' in step_custom:  # dungeon/map ID
                    metadata['d'] = step_custom['d']
                if 'l' in step_custom:  # level requirement
                    metadata['l'] = step_custom['l']
                
                if metadata:
                    step_metadata[step_id] = metadata
    
    print(f"🔗 Found custom relationships:")
    print(f"   Quest-to-step mappings: {len(quest_step_mappings)}")
    print(f"   Step-to-objective mappings: {len(step_objective_mappings)}")
    print(f"   Steps with metadata: {len(step_metadata)}")
    
    # Remove localization strings from all sections
    print(f"🧹 Removing localization strings...")
    
    # Clean Q.q (Quests)
    if 'Q.q' in parsed_data and 'items' in parsed_data['Q.q']:
        cleaned_quests = {}
        for quest_id, quest_data in parsed_data['Q.q']['items'].items():
            cleaned_quest = remove_localization_strings(quest_data)
            if cleaned_quest:  # Only keep non-empty quests
                cleaned_quests[quest_id] = cleaned_quest
        parsed_data['Q.q']['items'] = cleaned_quests
    
    # Clean Q.s (Steps)  
    if 'Q.s' in parsed_data and 'items' in parsed_data['Q.s']:
        cleaned_steps = {}
        for step_id, step_data in parsed_data['Q.s']['items'].items():
            cleaned_step = remove_localization_strings(step_data)
            if cleaned_step:  # Only keep non-empty steps
                cleaned_steps[step_id] = cleaned_step
        parsed_data['Q.s']['items'] = cleaned_steps
    
    # Clean Q.o (Objectives)
    if 'Q.o' in parsed_data and 'items' in parsed_data['Q.o']:
        cleaned_objectives = {}
        for obj_id, obj_data in parsed_data['Q.o']['items'].items():
            cleaned_obj = remove_localization_strings(obj_data)
            if cleaned_obj:  # Only keep non-empty objectives
                cleaned_objectives[obj_id] = cleaned_obj
        parsed_data['Q.o']['items'] = cleaned_objectives
    
    # Clean Q.t (Quest Objective Types) - Replace string values with null
    if 'Q.t' in parsed_data and 'items' in parsed_data['Q.t']:
        cleaned_types = {}
        for type_id, type_value in parsed_data['Q.t']['items'].items():
            # Replace all string values with null (these are localization templates)
            if isinstance(type_value, str):
                cleaned_types[type_id] = None
            else:
                cleaned_types[type_id] = type_value
        parsed_data['Q.t']['items'] = cleaned_types
    
    # Apply quest-to-step relationships
    quests_updated = 0
    if 'Q.q' in parsed_data and 'items' in parsed_data['Q.q']:
        for quest_id in quest_step_mappings:
            if quest_id in parsed_data['Q.q']['items']:
                quest_data = parsed_data['Q.q']['items'][quest_id]
                if not isinstance(quest_data, dict):
                    parsed_data['Q.q']['items'][quest_id] = {}
                    quest_data = parsed_data['Q.q']['items'][quest_id]
                
                # Add step IDs
                quest_data['s'] = quest_step_mappings[quest_id]
                quests_updated += 1
    
    # Apply step-to-objective relationships and metadata
    steps_updated = 0
    if 'Q.s' in parsed_data and 'items' in parsed_data['Q.s']:
        for step_id in step_objective_mappings:
            if step_id in parsed_data['Q.s']['items']:
                step_data = parsed_data['Q.s']['items'][step_id]
                if not isinstance(step_data, dict):
                    parsed_data['Q.s']['items'][step_id] = {}
                    step_data = parsed_data['Q.s']['items'][step_id]
                
                # Add objective IDs
                step_data['o'] = step_objective_mappings[step_id]
                
                # Add metadata (dungeon ID, level, etc.)
                if step_id in step_metadata:
                    step_data.update(step_metadata[step_id])
                
                steps_updated += 1
    
    print(f"✅ Applied relationships:")
    print(f"   Updated quests: {quests_updated}")
    print(f"   Updated steps: {steps_updated}")
    
    # Count final statistics
    final_quests = len([q for q in parsed_data['Q.q']['items'].values() if q])
    final_steps = len([s for s in parsed_data['Q.s']['items'].values() if s])
    final_objectives = len([o for o in parsed_data['Q.o']['items'].values() if o])
    final_types = len(parsed_data.get('Q.t', {}).get('items', {}))
    
    print(f"🧹 Localization cleanup results:")
    print(f"   Final quests (non-empty): {final_quests}")
    print(f"   Final steps (non-empty): {final_steps}")
    print(f"   Final objectives (non-empty): {final_objectives}")
    print(f"   Final objective types: {final_types} (strings → null)")
    
    # Save the enriched data
    output_file = "quests_fr_1248_enriched.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(parsed_data, f, indent=2, ensure_ascii=False)
    
    print(f"💾 Saved enriched data to: {output_file}")
    
    # Verify the results
    print("\n🔍 Verification - Sample enriched quest (no localization):")
    
    # Find a quest that should have been enriched
    sample_quest_id = None
    for quest_id in quest_step_mappings:
        if quest_id in parsed_data['Q.q']['items'] and parsed_data['Q.q']['items'][quest_id]:
            sample_quest_id = quest_id
            break
    
    if sample_quest_id:
        sample_quest = parsed_data['Q.q']['items'][sample_quest_id]
        print(f"Quest {sample_quest_id}: {sample_quest}")
        
        if 's' in sample_quest:
            for step_id in sample_quest['s'][:2]:  # Show first 2 steps
                step_id_str = str(step_id)
                if step_id_str in parsed_data['Q.s']['items']:
                    step_data = parsed_data['Q.s']['items'][step_id_str]
                    print(f"  Step {step_id}: {step_data}")
    
    print(f"\n🎉 Success! Created non-localized quest structure with:")
    print(f"   ✅ Quest-to-step relationships")
    print(f"   ✅ Step-to-objective relationships") 
    print(f"   ✅ Non-localized metadata only")
    print(f"   ✅ All text strings removed for separate localization loading")
    
    return output_file

# Run the extraction and merge
if __name__ == "__main__":
    print("=== EXTRACTING QUEST RELATIONSHIPS FROM CUSTOM CONFIG ===\n")
    result_file = extract_and_merge_quest_relationships()
    
    if result_file:
        print(f"\n🎉 Success! Enriched quest data saved to: {result_file}")
        print("\nThis enriched file now contains:")
        print("✅ Original localized text from ActionScript")
        print("✅ Quest-to-step relationships (s: [step_ids])")
        print("✅ Step-to-objective relationships (o: [objective_ids])")
        print("✅ Non-localized metadata (dungeon IDs, levels, etc.)")
        print("\nYou can now use this enriched file with the multilingual database builder.")

# Building the multilingual quests database

In [ ]:
def build_multilingual_quest_database():
    """
    Build a multilingual quest database that maintains Cyberia's QuestData structure
    but with multilingual text fields: Name, Description, Text become {lang: text, ...}
    """
    # Use the notebook-level output_path defined in the first cell
    global output_path
    # Ensure output_path is a Path object and exists
    if output_path is None:
        output_path = Path("output")
    else:
        # Accept strings or Path objects
        output_path = Path(output_path)
    if not output_path.exists():
        print(f"Output path does not exist: {output_path}")
        output_path.mkdir(parents=True, exist_ok=True)
    
    # Dynamically discover language codes from quest files
    quest_files = list(output_path.glob("quests_*_*.json"))
    language_data = {}
    
    print("Discovering quest language files...")
    
    # Extract language codes from found files
    for quest_file in quest_files:
        # Parse filename pattern: quests_{lang_code}_{version}.json
        filename_parts = quest_file.stem.split('_')
        if len(filename_parts) >= 3 and filename_parts[0] == 'quests':
            lang_code = filename_parts[1]
            version = filename_parts[2]
            
            print(f"Found quest file for language '{lang_code}': {quest_file.name}")
            
            try:
                with open(quest_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                language_data[lang_code] = data
                quest_count = len(data.get('Q.q', {}).get('items', {}))
                print(f"✓ Loaded {lang_code}: {quest_count} quests")
            except Exception as e:
                print(f"✗ Error loading {lang_code}: {e}")
        else:
            print(f"⚠ Skipping file with unexpected pattern: {quest_file.name}")
    
    if not language_data:
        print("No quest language files found! Expected pattern: quests_{lang}_{version}.json")
        return []
    
    discovered_languages = sorted(language_data.keys())
    print(f"\n✓ Discovered languages: {discovered_languages}")
    
    # Load auxiliary data for placeholder replacement
    auxiliary_data = {}
    
    print("Loading auxiliary data for placeholder replacement...")
    
    # Load NPC data for each language
    for lang_code in discovered_languages:
        npc_files = list(output_path.glob(f"npc_{lang_code}_*.json"))
        items_files = list(output_path.glob(f"items_{lang_code}_*.json"))
        monsters_files = list(output_path.glob(f"monsters_{lang_code}_*.json"))
        
        auxiliary_data[lang_code] = {
            'npcs': {},
            'items': {},
            'monsters': {}
        }
        
        # Load NPCs
        for npc_file in npc_files:
            try:
                with open(npc_file, 'r', encoding='utf-8') as f:
                    npc_data = json.load(f)
                    npcs = npc_data.get('N.d', {}).get('items', {})
                    auxiliary_data[lang_code]['npcs'].update(npcs)
                    print(f"✓ Loaded {len(npcs)} NPCs for {lang_code}")
                    break  # Use first file found
            except Exception as e:
                print(f"✗ Error loading NPCs for {lang_code}: {e}")
        
        # Load Items
        for items_file in items_files:
            try:
                with open(items_file, 'r', encoding='utf-8') as f:
                    items_data = json.load(f)
                    items = items_data.get('I.u', {}).get('items', {})
                    auxiliary_data[lang_code]['items'].update(items)
                    print(f"✓ Loaded {len(items)} items for {lang_code}")
                    break  # Use first file found
            except Exception as e:
                print(f"✗ Error loading items for {lang_code}: {e}")
        
        # Load Monsters
        for monsters_file in monsters_files:
            try:
                with open(monsters_file, 'r', encoding='utf-8') as f:
                    monsters_data = json.load(f)
                    monsters = monsters_data.get('M', {}).get('items', {})
                    auxiliary_data[lang_code]['monsters'].update(monsters)
                    print(f"✓ Loaded {len(monsters)} monsters for {lang_code}")
                    break  # Use first file found
            except Exception as e:
                print(f"✗ Error loading monsters for {lang_code}: {e}")
    
    def replace_quest_placeholders(template, type_id, parameters, lang_code):
        """
        Replace placeholders in quest objective templates with actual localized values
        """
        if not template or not isinstance(template, str):
            return template
        
        # Clean template
        text = template.strip('"')
        
        if not parameters or not isinstance(parameters, list):
            return text
        
        try:
            aux_data = auxiliary_data.get(lang_code, {})
            npcs = aux_data.get('npcs', {})
            items = aux_data.get('items', {})
            monsters = aux_data.get('monsters', {})
            
            # Handle different quest objective types
            if type_id == 1:  # "Aller voir #1" - Go see NPC
                if len(parameters) >= 1:
                    npc_id = str(parameters[0])
                    npc_name = npcs.get(npc_id, {}).get('n', f'NPC {npc_id}')
                    text = text.replace('#1', npc_name)
            
            elif type_id == 9:  # "Retourner voir #1" - Return to see NPC
                if len(parameters) >= 1:
                    npc_id = str(parameters[0])
                    npc_name = npcs.get(npc_id, {}).get('n', f'NPC {npc_id}')
                    text = text.replace('#1', npc_name)
            
            elif type_id == 4:  # "Découvrir la carte #1" - Discover map
                if len(parameters) >= 1:
                    # For type 4, the parameter is usually a map/area description
                    map_description = str(parameters[0])
                    text = text.replace('#1', map_description)
            
            elif type_id == 2:  # "Montrer à #1 : #3 #2" - Show to NPC: quantity item
                if len(parameters) >= 3:
                    npc_id = str(parameters[0])
                    item_id = str(parameters[1])
                    quantity = str(parameters[2])
                    
                    npc_name = npcs.get(npc_id, {}).get('n', f'NPC {npc_id}')
                    item_data = items.get(item_id, {})
                    item_name = item_data.get('n', f'Item {item_id}') if isinstance(item_data, dict) else f'Item {item_id}'
                    
                    text = text.replace('#1', npc_name)
                    text = text.replace('#2', item_name)
                    text = text.replace('#3', quantity)
            
            elif type_id == 3:  # "Ramener à #1 : x#3 #2" - Bring to NPC: x quantity item
                if len(parameters) >= 3:
                    npc_id = str(parameters[0])
                    item_id = str(parameters[1])
                    quantity = str(parameters[2])
                    
                    npc_name = npcs.get(npc_id, {}).get('n', f'NPC {npc_id}')
                    item_data = items.get(item_id, {})
                    item_name = item_data.get('n', f'Item {item_id}') if isinstance(item_data, dict) else f'Item {item_id}'
                    
                    text = text.replace('#1', npc_name)
                    text = text.replace('#2', item_name)
                    text = text.replace('#3', quantity)
            
            elif type_id == 12:  # "Rapporter #3 âme de #2 à #1" - Bring quantity soul of item to NPC
                if len(parameters) >= 3:
                    npc_id = str(parameters[0])
                    item_id = str(parameters[1])
                    quantity = str(parameters[2])
                    
                    npc_name = npcs.get(npc_id, {}).get('n', f'NPC {npc_id}')
                    item_data = items.get(item_id, {})
                    item_name = item_data.get('n', f'Item {item_id}') if isinstance(item_data, dict) else f'Item {item_id}'
                    
                    text = text.replace('#1', npc_name)
                    text = text.replace('#2', item_name)
                    text = text.replace('#3', quantity)
            
            elif type_id == 6:  # "Vaincre x#2 #1 en un seul combat" - Defeat x quantity monster in single combat
                if len(parameters) >= 2:
                    monster_id = str(parameters[0])
                    quantity = str(parameters[1])
                    
                    monster_name = monsters.get(monster_id, {}).get('n', f'Monster {monster_id}')
                    
                    text = text.replace('#1', monster_name)
                    text = text.replace('#2', quantity)
            
            elif type_id == 16:  # "Vaincre #2 monstres de la famille #1 en un seul combat"
                if len(parameters) >= 2:
                    family_name = str(parameters[0])
                    quantity = str(parameters[1])
                    
                    text = text.replace('#1', family_name)
                    text = text.replace('#2', quantity)
            
            elif type_id == 18:  # "Ramener à #1 #4 #3 dans la zone #2"
                if len(parameters) >= 4:
                    npc_id = str(parameters[0])
                    zone_name = str(parameters[1])
                    item_id = str(parameters[2])
                    quantity = str(parameters[3])
                    
                    npc_name = npcs.get(npc_id, {}).get('n', f'NPC {npc_id}')
                    item_data = items.get(item_id, {})
                    item_name = item_data.get('n', f'Item {item_id}') if isinstance(item_data, dict) else f'Item {item_id}'
                    
                    text = text.replace('#1', npc_name)
                    text = text.replace('#2', zone_name)
                    text = text.replace('#3', item_name)
                    text = text.replace('#4', quantity)
            
            else:
                # Generic replacement for other types
                for i, param in enumerate(parameters):
                    placeholder = f"#{i + 1}"
                    if placeholder in text:
                        text = text.replace(placeholder, str(param))
        
        except Exception as e:
            print(f"Warning: Error replacing placeholders for type {type_id} in {lang_code}: {e}")
            # Fallback to basic replacement
            for i, param in enumerate(parameters):
                placeholder = f"#{i + 1}"
                if placeholder in text:
                    text = text.replace(placeholder, str(param))
        
        return text

    # Use "fr" as primary language, otherwise the first available language as the primary for structure
    primary_lang = "fr" if "fr" in discovered_languages else discovered_languages[0]
    print(f"Using '{primary_lang}' as primary language for structure")
    
    primary_data = language_data[primary_lang]
    
    # Extract data from primary language
    all_quest_data = primary_data.get('Q.q', {}).get('items', {})
    all_step_data = primary_data.get('Q.s', {}).get('items', {})
    all_objective_data = primary_data.get('Q.o', {}).get('items', {})
    all_objective_types = primary_data.get('Q.t', {}).get('items', {})
    
    # Load custom data for mappings
    step_objective_mappings = {}

    def download_custom_data():
        print("Custom data not found. Downloading from GitHub...")
        custom_api_dir = Path("api/custom")
        # The user stated the directory exists, but we can ensure it safely.
        custom_api_dir.mkdir(parents=True, exist_ok=True)
        
        # Use GitHub API to list files in the directory
        api_url = "https://api.github.com/repos/Lounek09/Cyberia/contents/Cyberia.Api/api/custom"
        
        try:
            response = requests.get(api_url, timeout=30)
            response.raise_for_status()
            files_to_download = response.json()
            
            for file_info in files_to_download:
                if file_info['type'] == 'file':
                    filename = file_info['name']
                    download_url = file_info['download_url']
                    dest_path = custom_api_dir / filename
                    
                    if dest_path.exists():
                        print(f"✓ {filename} already exists. Skipping download.")
                        continue
                    
                    try:
                        print(f"Downloading {download_url}...")
                        file_response = requests.get(download_url, timeout=30)
                        file_response.raise_for_status()
                        with open(dest_path, 'w', encoding='utf-8') as f:
                            f.write(file_response.text)
                        print(f"✓ Saved to {dest_path}")
                    except requests.exceptions.RequestException as e:
                        print(f"✗ Failed to download {filename}: {e}")

        except requests.exceptions.RequestException as e:
            print(f"✗ Failed to list files from GitHub API: {e}")


    custom_data_path = Path("api/custom/quests.json")
    try:
        # Attempt to open the file first
        with open(custom_data_path, 'r', encoding='utf-8-sig') as f:
            custom_data = json.load(f)
        print("✓ Loading custom data for step-to-objective mappings...")
    except FileNotFoundError:
        # If it fails, download the files and try again
        download_custom_data()
        try:
            with open(custom_data_path, 'r', encoding='utf-8-sig') as f:
                custom_data = json.load(f)
            print("✓ Loading custom data for step-to-objective mappings after download...")
        except Exception as e:
            print(f"Error loading custom data after download: {e}")
            custom_data = {} # Ensure custom_data exists
    except Exception as e:
        print(f"Error loading custom data: {e}")
        custom_data = {} # Ensure custom_data exists

    if 'CQ.s' in custom_data:
        for step_custom in custom_data['CQ.s']:
            step_id = str(step_custom.get('id'))
            objective_ids = step_custom.get('o', [])
            if step_id and objective_ids:
                step_objective_mappings[step_id] = objective_ids
        print(f"✓ Loaded mappings for {len(step_objective_mappings)} steps")
    
    def get_multilingual_text(item_id, section, field, language_data):
        """Extract multilingual text for a specific field"""
        multilingual_text = {}
        
        for lang_code, lang_data in language_data.items():
            try:
                items = lang_data.get(section, {}).get('items', {})
                if item_id in items:
                    item = items[item_id]
                    if isinstance(item, dict) and field in item:
                        text = item[field]
                        if text:  # Only add non-empty text
                            multilingual_text[lang_code] = text
            except Exception as e:
                continue
        
        return multilingual_text
    
    def get_multilingual_objective_text(obj_id, type_id, parameters, language_data):
        """Format multilingual objective text using type templates with proper placeholder replacement"""
        multilingual_text = {}
        
        # Special handling for TypeId 0 - these are custom objectives with text in parameters
        if type_id == 0:
            # For TypeId 0, look up the same objective ID in each language file
            for lang_code, lang_data in language_data.items():
                try:
                    objectives = lang_data.get('Q.o', {}).get('items', {})
                    obj_id_str = str(obj_id)
                    
                    if obj_id_str in objectives:
                        obj_data = objectives[obj_id_str]
                        if (isinstance(obj_data, dict) and 
                            obj_data.get('t', 0) == 0 and 
                            obj_data.get('p') and 
                            isinstance(obj_data.get('p'), list) and 
                            len(obj_data.get('p')) > 0):
                            
                            # Use the localized text from this language's version
                            localized_text = str(obj_data.get('p')[0])
                            # Clean up any escaped quotes
                            localized_text = localized_text.replace('\\"', '"').replace('\"', '"')
                            multilingual_text[lang_code] = localized_text
                        else:
                            # Fallback to parameters if structure is different
                            if parameters and isinstance(parameters, list) and len(parameters) > 0:
                                clean_text = str(parameters[0]).replace('\\"', '"').replace('\"', '"')
                                multilingual_text[lang_code] = clean_text
                    else:
                        # Objective not found in this language, use parameters as fallback
                        if parameters and isinstance(parameters, list) and len(parameters) > 0:
                            clean_text = str(parameters[0]).replace('\\"', '"').replace('\"', '"')
                            multilingual_text[lang_code] = clean_text
                            
                except Exception as e:
                    # Fallback to parameters
                    if parameters and isinstance(parameters, list) and len(parameters) > 0:
                        clean_text = str(parameters[0]).replace('\\"', '"').replace('\"', '"')
                        multilingual_text[lang_code] = clean_text
                    else:
                        multilingual_text[lang_code] = "Custom objective"
                    continue
        else:
            # Regular template-based objectives (TypeId != 0) with enhanced placeholder replacement
            for lang_code, lang_data in language_data.items():
                try:
                    # Get the localized parameters for this specific objective in this language
                    objectives = lang_data.get('Q.o', {}).get('items', {})
                    obj_id_str = str(obj_id)
                    localized_parameters = parameters  # Default fallback
                    
                    if obj_id_str in objectives:
                        obj_data = objectives[obj_id_str]
                        if isinstance(obj_data, dict) and 'p' in obj_data:
                            localized_parameters = obj_data.get('p', parameters)
                    
                    # Get the template for this language
                    objective_types = lang_data.get('Q.t', {}).get('items', {})
                    if str(type_id) in objective_types:
                        template = objective_types[str(type_id)]
                        if isinstance(template, str):
                            # Use the localized parameters with the template
                            formatted_text = replace_quest_placeholders(template, type_id, localized_parameters, lang_code)
                            multilingual_text[lang_code] = formatted_text
                except Exception as e:
                    print(f"Warning: Error processing objective text for {lang_code}, type {type_id}: {e}")
                    continue
        
        return multilingual_text
    
    # Continue with rest of the existing function...
    def parse_rewards(reward_data, language_data=None, auxiliary_data=None):
        """Parse reward array into QuestRewardData structure with multilingual item names"""
        if not reward_data or not isinstance(reward_data, list):
            return {
                "ItemsReward": [],
                "EmotesReward": [],
                "ExperienceReward": 0,
                "KamasReward": 0
            }
        
        rewards = {
            "ItemsReward": [],
            "EmotesReward": [],
            "ExperienceReward": 0,
            "KamasReward": 0
        }
        
        if len(reward_data) > 0 and reward_data[0] is not None:
            rewards["KamasReward"] = reward_data[0]
        
        if len(reward_data) > 1 and reward_data[1] is not None:
            rewards["ExperienceReward"] = reward_data[1]
        
        if len(reward_data) > 2 and reward_data[2] is not None and isinstance(reward_data[2], list):
            for item_data in reward_data[2]:
                if isinstance(item_data, list) and len(item_data) >= 2:
                    item_id = item_data[0]
                    quantity = item_data[1]
                    
                    # Get multilingual item names
                    item_names = {}
                    if auxiliary_data:
                        for lang_code, aux_data in auxiliary_data.items():
                            items = aux_data.get('items', {})
                            if str(item_id) in items:
                                item_info = items[str(item_id)]
                                if isinstance(item_info, dict) and 'n' in item_info:
                                    item_names[lang_code] = item_info['n']
                    
                    reward_item = {
                        "ItemId": item_id,
                        "Quantity": quantity
                    }
                    
                    # Add multilingual names if available
                    if item_names:
                        reward_item["Name"] = item_names
                    
                    rewards["ItemsReward"].append(reward_item)
        
        if len(reward_data) > 3 and reward_data[3] is not None:
            rewards["EmotesReward"] = reward_data[3] if isinstance(reward_data[3], list) else []
        
        return rewards
    
    def get_step_objectives(step_id, mappings, language_data):
        """Get multilingual objectives for a step"""
        objectives = []
        step_id_str = str(step_id)
        
        if step_id_str in mappings:
            objective_ids = mappings[step_id_str]
            
            for obj_id in objective_ids:
                obj_id_str = str(obj_id)
                # Get objective data from primary language (first available)
                primary_lang = list(language_data.keys())[0]
                primary_objectives = language_data[primary_lang].get('Q.o', {}).get('items', {})
                
                if obj_id_str in primary_objectives:
                    obj_data = primary_objectives[obj_id_str]
                    
                    objective = {
                        "Id": int(obj_id),
                        "TypeId": obj_data.get('t', 0),
                        "Text": get_multilingual_objective_text(
                            obj_id,  # Pass the objective ID for TypeId=0 lookup
                            obj_data.get('t', 0), 
                            obj_data.get('p', []), 
                            language_data
                        ),
                        "X": obj_data.get('x', 0),
                        "Y": obj_data.get('y', 0),
                        "Parameters": obj_data.get('p', [])
                    }
                    objectives.append(objective)
        
        return objectives
    
    # Build multilingual quest database
    print(f"\nBuilding multilingual quest database with enhanced placeholder replacement...")
    quest_database = []
    processed_count = 0
    mapped_steps_count = 0
    typeid_0_objectives = 0
    
    # Track potential duplicates and TypeId=0 statistics
    name_groups = {}
    
    for quest_id, quest_info in all_quest_data.items():
        try:
            if not isinstance(quest_info, dict):
                continue
                
            quest_obj = {
                "Id": int(quest_id),
                "Name": get_multilingual_text(quest_id, 'Q.q', 'n', language_data),
                "CategoryId": quest_info.get("c", 0),
                "Steps": [],
                "Rewards": {
                    "ItemsReward": [],
                    "EmotesReward": [],
                    "ExperienceReward": 0,
                    "KamasReward": 0
                },
                "has_dungeon": False
            }
            
            # Build multilingual steps
            step_ids = quest_info.get("s", [])
            if isinstance(step_ids, list):
                for step_id in step_ids:
                    step_id_str = str(step_id)
                    if step_id_str in all_step_data:
                        step_data = all_step_data[step_id_str]
                        
                        if isinstance(step_data, dict):
                            objectives = get_step_objectives(step_id, step_objective_mappings, language_data)
                            
                            # Count TypeId=0 objectives
                            for obj in objectives:
                                if obj["TypeId"] == 0:
                                    typeid_0_objectives += 1
                            
                            step_obj = {
                                "Id": int(step_id),
                                "Name": get_multilingual_text(step_id_str, 'Q.s', 'n', language_data),
                                "Description": get_multilingual_text(step_id_str, 'Q.s', 'd', language_data),
                                "OptimalLevel": 1,
                                "Objectives": objectives
                            }
                            
                            if objectives:
                                mapped_steps_count += 1
                            
                            quest_obj["Steps"].append(step_obj)
                            
                            # Use step rewards as quest rewards (first step)
                            if len(quest_obj["Steps"]) == 1:
                                step_rewards = step_data.get("r", [])
                                quest_obj["Rewards"] = parse_rewards(step_rewards, language_data, auxiliary_data)
            
            # Only include quests with non-empty names in at least one language
            if quest_obj["Name"] and quest_obj["Steps"]:
                quest_database.append(quest_obj)
                processed_count += 1
                
                # Track quests with same name for duplicate analysis
                if quest_obj["Name"]:
                    # Use the first available language as the key
                    primary_name = next(iter(quest_obj["Name"].values()))
                    if primary_name not in name_groups:
                        name_groups[primary_name] = []
                    name_groups[primary_name].append({
                        "id": quest_obj["Id"],
                        "steps": [step["Id"] for step in quest_obj["Steps"]],
                        "multilingual_name": quest_obj["Name"]
                    })
            
        except Exception as e:
            print(f"Error processing quest {quest_id}: {e}")
            continue
    
    print(f"\n✓ Built multilingual quest database with enhanced placeholder replacement")
    print(f"✓ Processed {processed_count} quests with properly localized objective text")
    print(f"✓ Steps with proper objective mappings: {mapped_steps_count}")
    print(f"✓ TypeId=0 objectives (custom text): {typeid_0_objectives}")
    print(f"✓ Languages: {discovered_languages}")
    
    return quest_database

# Build the multilingual database
print("=== BUILDING MULTILINGUAL QUEST DATABASE ===\n")

multilingual_quest_database = build_multilingual_quest_database()

if multilingual_quest_database:
    # Save the multilingual database
    multilingual_output_file = "Quests_database_multilingual.json"
    with open(multilingual_output_file, 'w', encoding='utf-8') as f:
        json.dump(multilingual_quest_database, f, indent=2, ensure_ascii=False)
    
    print(f"\n✓ Multilingual quest database saved to {multilingual_output_file}")
    
    # Show sample multilingual quest
    print(f"\n=== SAMPLE MULTILINGUAL QUEST ===")
    if multilingual_quest_database:
        sample = multilingual_quest_database[0]
        print(f"Quest ID: {sample['Id']}")
        print("Multilingual Names:")
        for lang, name in sample['Name'].items():
            print(f"  {lang}: {name}")
        
        if sample['Steps']:
            step = sample['Steps'][0]
            print(f"\nStep ID: {step['Id']}")
            print("Multilingual Step Names:")
            for lang, name in step['Name'].items():
                print(f"  {lang}: {name}")
            
            print("Multilingual Descriptions:")
            for lang, desc in step['Description'].items():
                print(f"  {lang}: {desc[:50]}...")
            
            if step['Objectives']:
                obj = step['Objectives'][0]
                print(f"\nObjective ID: {obj['Id']}, TypeId: {obj['TypeId']}")
                print("Multilingual Objective Text:")
                for lang, text in obj['Text'].items():
                    print(f"  {lang}: {text}")
    
    # Show statistics with discovered languages
    print(f"\n=== MULTILINGUAL DATABASE STATISTICS ===")
    total_quests = len(multilingual_quest_database)
    total_steps = sum(len(quest["Steps"]) for quest in multilingual_quest_database)
    total_objectives = sum(len(step["Objectives"]) for quest in multilingual_quest_database for step in quest["Steps"])
    
    # Count TypeId=0 objectives and language coverage
    typeid_0_count = sum(1 for quest in multilingual_quest_database 
                        for step in quest["Steps"] 
                        for obj in step["Objectives"] 
                        if obj["TypeId"] == 0)
    
    quest_name_coverage = {}
    step_name_coverage = {}
    objective_text_coverage = {}
    
    for quest in multilingual_quest_database:
        for lang in quest['Name'].keys():
            quest_name_coverage[lang] = quest_name_coverage.get(lang, 0) + 1
        
        for step in quest['Steps']:
            for lang in step['Name'].keys():
                step_name_coverage[lang] = step_name_coverage.get(lang, 0) + 1
            
            for obj in step['Objectives']:
                for lang in obj['Text'].keys():
                    objective_text_coverage[lang] = objective_text_coverage.get(lang, 0) + 1
    
    print(f"Total quests: {total_quests}")
    print(f"Total steps: {total_steps}")
    print(f"Total objectives: {total_objectives}")
    print(f"TypeId=0 objectives (custom text): {typeid_0_count}")
    
    print(f"\nLanguage Coverage:")
    print("Quest Names:")
    for lang, count in sorted(quest_name_coverage.items()):
        percentage = (count / total_quests) * 100
        print(f"  {lang}: {count}/{total_quests} ({percentage:.1f}%)")
    
    print("Step Names:")
    for lang, count in sorted(step_name_coverage.items()):
        percentage = (count / total_steps) * 100 if total_steps > 0 else 0
        print(f"  {lang}: {count}/{total_steps} ({percentage:.1f}%)")
    
    print("Objective Text:")
    for lang, count in sorted(objective_text_coverage.items()):
        percentage = (count / total_objectives) * 100 if total_objectives > 0 else 0
        print(f"  {lang}: {count}/{total_objectives} ({percentage:.1f}%)")
    
else:
    print("❌ Failed to build multilingual quest database")

print(f"\n=== STRUCTURE VERIFICATION ===")
print("✓ Maintains Cyberia's QuestData structure")
print("✓ Text fields are now multilingual dictionaries")
print("✓ Preserves all numeric IDs and relationships")
print("✓ Compatible with existing Cyberia API expectations")
print("✓ Dynamically discovers available languages")
print("✓ Properly handles TypeId=0 objectives with localized text")

=== BUILDING MULTILINGUAL QUEST DATABASE ===

Discovering quest language files...
Found quest file for language 'en': quests_en_1248.json
✓ Loaded en: 841 quests
Found quest file for language 'es': quests_es_1248.json
✓ Loaded es: 871 quests
Found quest file for language 'fr': quests_fr_1248.json
✓ Loaded fr: 815 quests
Found quest file for language 'pt': quests_pt_1248.json
✓ Loaded pt: 890 quests

✓ Discovered languages: ['en', 'es', 'fr', 'pt']
Loading auxiliary data for placeholder replacement...
✓ Loaded 1111 NPCs for en
✓ Loaded 11541 items for en
✓ Loaded 1568 monsters for en
✓ Loaded 1119 NPCs for es
✓ Loaded 11615 items for es
✓ Loaded 1587 monsters for es
✓ Loaded 1011 NPCs for fr
✓ Loaded 11415 items for fr
✓ Loaded 1450 monsters for fr
✓ Loaded 1124 NPCs for pt
✓ Loaded 11654 items for pt
✓ Loaded 1604 monsters for pt
Using 'en' as primary language for structure
✓ Loading custom data for step-to-objective mappings...
✓ Loaded mappings for 825 steps

Building multilingual qu

In [15]:
def create_quest_database_html():
    """
    Create an advanced responsive HTML file to display the quest database hierarchy
    with multilingual support and advanced text search features
    """
    
    # Load the multilingual quest database
    try:
        with open("Quests_database_multilingual.json", 'r', encoding='utf-8') as f:
            quest_data = json.load(f)
        print(f"✓ Loaded {len(quest_data)} quests from multilingual database")
    except Exception as e:
        print(f"✗ Error loading quest database: {e}")
        return
    
    # Analyze available languages
    all_languages = set()
    for quest in quest_data:
        if quest.get('Name'):
            all_languages.update(quest['Name'].keys())
        for step in quest.get('Steps', []):
            if step.get('Name'):
                all_languages.update(step['Name'].keys())
            for obj in step.get('Objectives', []):
                if obj.get('Text'):
                    all_languages.update(obj['Text'].keys())
        # Check item rewards for languages
        for item in quest.get('Rewards', {}).get('ItemsReward', []):
            if item.get('Name'):
                all_languages.update(item['Name'].keys())
    
    available_languages = sorted(all_languages)
    default_language = available_languages[0] if available_languages else 'en'
    
    print(f"✓ Available languages: {available_languages}")
    print(f"✓ Default language: {default_language}")
    
    # Analyze quest types and categories for advanced filtering
    categories = set(q.get('CategoryId', 0) for q in quest_data)
    objective_types = set()
    for quest in quest_data:
        for step in quest.get('Steps', []):
            for obj in step.get('Objectives', []):
                objective_types.add(obj.get('TypeId', 0))
    
    # Create HTML content
    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>RETRO Quest Database Viewer - Advanced Search</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}

        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }}

        /* Courier New font override */
        body.courier-font {{
            font-family: 'Courier New', 'Consolas', monospace;
        }}

        body.courier-font * {{
            font-family: 'Courier New', 'Consolas', monospace;
        }}

        .container {{
            max-width: 1600px;
            margin: 0 auto;
            background: rgba(255, 255, 255, 0.95);
            border-radius: 15px;
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
            overflow: hidden;
            position: relative;
        }}

        .header {{
            background: linear-gradient(135deg, #2c3e50, #34495e);
            color: white;
            padding: 30px;
            text-align: center;
            position: sticky;
            top: 0;
            z-index: 100;
            height: 240px;
        }}

        .header h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
        }}

        .stats {{
            display: flex;
            justify-content: center;
            gap: 30px;
            margin-top: 20px;
            flex-wrap: wrap;
        }}

        .stat-item {{
            background: rgba(255, 255, 255, 0.1);
            padding: 15px 25px;
            border-radius: 10px;
            text-align: center;
        }}

        .stat-number {{
            font-size: 2em;
            font-weight: bold;
            display: block;
        }}

        .search-container {{
            max-width: 1200px;
            margin: 0 auto 0px;
            background: white;
            padding: 20px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            position: sticky;
            top: 170px;
            z-index: 99;
            height: auto;
        }}

        .search-box input {{
            width: 100%;
            padding: 12px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 8px;
            margin-bottom: 15px;
        }}

        .search-options-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            margin-bottom: 15px;
        }}

        .option-group {{
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            border: 1px solid #e9ecef;
        }}

        .option-group h4 {{
            margin: 0 0 15px 0;
            color: #495057;
            font-size: 14px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}

        .option-table {{
            display: flex;
            flex-direction: column;
            gap: 8px;
        }}

        .option-row {{
            display: flex;
            align-items: flex-start;
            padding: 8px 12px;
            border-radius: 4px;
            transition: background-color 0.2s ease;
            cursor: pointer;
            margin: 0;
            min-height: 32px;
            gap: 8px;
        }}

        .option-row:hover {{
            background-color: rgba(76, 175, 80, 0.1);
        }}

        .option-row input[type="radio"],
        .option-row input[type="checkbox"] {{
            margin: 0;
            flex-shrink: 0;
            width: 16px;
            height: 16px;
            margin-top: 2px;
        }}

        .option-text {{
            flex: 1;
            font-size: 14px;
            line-height: 1.4;
            word-wrap: break-word;
            overflow-wrap: break-word;
            max-width: calc(100% - 32px);
        }}

        .language-highlight-row {{
            justify-content: space-between;
            align-items: center;
        }}

        .language-highlight-row .option-text {{
            flex: 0 0 auto;
            margin-right: 10px;
        }}

        .language-dropdown {{
            padding: 4px 8px;
            border: 1px solid #ddd;
            border-radius: 4px;
            font-size: 12px;
        }}

        .search-controls {{
            text-align: center;
            margin-top: 15px;
            display: flex;
            gap: 10px;
            justify-content: center;
            flex-wrap: wrap;
        }}

        .apply-btn, .control-btn {{
            background: linear-gradient(135deg, #4CAF50, #45a049);
            color: white;
            border: none;
            padding: 12px 24px;
            border-radius: 8px;
            font-size: 16px;
            cursor: pointer;
            transition: all 0.3s ease;
            display: inline-flex;
            align-items: center;
            gap: 8px;
        }}

        .control-btn {{
            background: linear-gradient(135deg, #2196F3, #1976D2);
            padding: 10px 16px;
            font-size: 14px;
        }}

        .toggle-filters {{
            background: linear-gradient(135deg, #ff9800, #f57c00);
        }}

        .apply-btn:hover, .control-btn:hover {{
            transform: translateY(-1px);
        }}

        .apply-btn.changed {{
            background: linear-gradient(135deg, #ff9800, #f57c00);
        }}

        .apply-btn.changed .btn-icon {{
            display: inline !important;
        }}

        .filters-hidden .search-options-grid {{
            display: none;
        }}

        .results-info {{
            padding: 10px 30px;
            background: #e3f2fd;
            border-bottom: 1px solid #bbdefb;
            color: #1976d2;
            font-weight: 500;
            text-align: center;
            margin: 0; /* Remove any margin */
            position: sticky;
            top: calc(170px + 200px); /* Position below search container */
            z-index: 98;
        }}

        .quest-list {{
            padding: 20px 30px 30px 30px;
            overflow-y: auto;
            position: relative;
        }}

        .quest-list.display-columns .language-variant::before {{
            display: none;
        }}

        .quest-list.display-rows .language-variant::before {{
            display: inline-block;
        }}

        .language-header-row {{
            display: flex;
            gap: 15px;
            margin: 0 0 15px 0;
            padding: 12px;
            border-bottom: 1px solid #dee2e6;
            position: fixed;
            top: 0;
            left: 0;
            right: 0;
            background: rgba(255, 255, 255, 0.95);
            z-index: 1000;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            border-radius: 0;
            border: none;
            backdrop-filter: blur(10px);
            transform: translateY(-100%);
            transition: transform 0.3s ease;
            max-width: 1600px;
            margin-left: auto;
            margin-right: auto;
        }}

        .language-header-row.floating {{
            transform: translateY(0);
        }}

        .language-header {{
            min-width: 120px;
            flex: 1;
            font-size: 0.85em;
            font-weight: 600;
            color: #495057;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}

        .language-header.hidden {{
            display: none;
        }}

        .quest-list.display-rows .language-header-row {{
            display: none !important;
        }}

        .quest-item {{
            background: white;
            border: 1px solid #e9ecef;
            border-radius: 10px;
            margin-bottom: 20px;
            overflow: hidden;
            transition: all 0.3s ease;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.05);
            display: none;
        }}

        .quest-item:hover {{
            transform: translateY(-2px);
            box-shadow: 0 5px 20px rgba(0, 0, 0, 0.1);
        }}

        .quest-item.highlight {{
            border-color: #ffc107;
            box-shadow: 0 0 15px rgba(255, 193, 7, 0.3);
        }}

        .quest-item:first-child {{
            margin-top: 0;
        }}

        .quest-header {{
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            padding: 20px;
            cursor: pointer;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }}

        .quest-header:hover {{
            background: linear-gradient(135deg, #5a6fd8, #6a4190);
        }}

        .quest-title {{
            font-size: 1.3em;
            font-weight: 600;
        }}

        .quest-meta {{
            font-size: 0.9em;
            opacity: 0.9;
            margin-top: 5px;
        }}

        .expand-icon {{
            font-size: 1.2em;
            transition: transform 0.3s;
        }}

        .quest-content {{
            display: none;
            padding: 0;
        }}

        .quest-content.expanded {{
            display: block;
            animation: slideDown 0.3s ease;
        }}

        .multilingual-content {{
            display: flex;
            flex-direction: row;
            gap: 15px;
            flex-wrap: wrap;
        }}

        .multilingual-content.display-rows {{
            flex-direction: column;
            gap: 5px;
        }}

        .language-variant {{
            display: none;
            padding: 8px 12px;
            background: rgba(255, 255, 255, 0.9);
            border-radius: 6px;
            border: 1px solid rgba(0, 0, 0, 0.2);
            min-width: 120px;
            flex: 1;
            color: #2c3e50;
            position: relative;
        }}

        .language-variant::before {{
            content: attr(data-lang-display) ": ";
            font-size: 0.75em;
            font-weight: 600;
            color: #6c757d;
            text-transform: uppercase;
            display: none;
            margin-right: 6px;
        }}

        .language-variant.visible {{
            display: block;
        }}

        .display-rows .language-variant {{
            min-width: auto;
            flex: none;
        }}

        .language-text {{
            display: inline;
        }}

        /* Language highlighting */
        .lang-row.highlighted-fr {{
            border: 2px dotted #e74c3c;
            padding: 4px;
            border-radius: 4px;
            background-color: rgba(231, 76, 60, 0.1);
        }}

        .lang-row.highlighted-en {{
            border: 2px dotted #3498db;
            padding: 4px;
            border-radius: 4px;
            background-color: rgba(52, 152, 219, 0.1);
        }}

        .lang-row.highlighted-es {{
            border: 2px dotted #f39c12;
            padding: 4px;
            border-radius: 4px;
            background-color: rgba(243, 156, 18, 0.1);
        }}

        .lang-row.highlighted-pt {{
            border: 2px dotted #27ae60;
            padding: 4px;
            border-radius: 4px;
            background-color: rgba(39, 174, 96, 0.1);
        }}

        .search-highlight {{
            background-color: yellow;
            font-weight: bold;
            padding: 1px 2px;
            border-radius: 2px;
        }}

        .null-value {{
            background-color: #ffebee;
            border: 1px solid #f8bbd9;
            border-radius: 4px;
            padding: 4px;
            color: #c62828;
            font-style: italic;
            display: inline-block;
        }}

        /* Back to top button */
        .back-to-top {{
            position: fixed;
            bottom: 30px;
            right: 30px;
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            width: 50px;
            height: 50px;
            border-radius: 50%;
            border: none;
            font-size: 20px;
            cursor: pointer;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.3);
            transition: all 0.3s ease;
            z-index: 1000;
            display: none;
        }}

        .back-to-top:hover {{
            transform: translateY(-2px);
            box-shadow: 0 6px 16px rgba(0, 0, 0, 0.4);
        }}

        .back-to-top.show {{
            display: block;
        }}

        @keyframes slideDown {{
            from {{ opacity: 0; max-height: 0; }}
            to {{ opacity: 1; max-height: 1000px; }}
        }}

        .step-item {{
            border-left: 4px solid #667eea;
            margin: 15px 20px;
            background: #f8f9fa;
            border-radius: 0 8px 8px 0;
        }}

        .step-header {{
            background: #e9ecef;
            padding: 15px 20px;
            font-weight: 600;
            color: #495057;
            cursor: pointer;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }}

        .step-header:hover {{
            background: #dee2e6;
        }}

        .step-content {{
            padding: 20px;
            display: none;
        }}

        .step-content.expanded {{
            display: block;
        }}

        .step-description {{
            margin-bottom: 15px;
            color: #6c757d;
            line-height: 1.6;
        }}

        .objectives-section {{
            margin-top: 15px;
        }}

        .objectives-title {{
            font-weight: 600;
            color: #495057;
            margin-bottom: 10px;
            display: flex;
            align-items: center;
            gap: 5px;
        }}

        .objective-item {{
            background: white;
            border: 1px solid #dee2e6;
            border-radius: 6px;
            padding: 12px 15px;
            margin-bottom: 8px;
            display: flex;
            align-items: center;
            gap: 10px;
        }}

        .objective-type {{
            background: #667eea;
            color: white;
            padding: 3px 8px;
            border-radius: 12px;
            font-size: 0.8em;
            font-weight: 600;
        }}

        .objective-coordinates {{
            background: #28a745;
            color: white;
            padding: 3px 8px;
            border-radius: 12px;
            font-size: 0.8em;
            margin-left: auto;
        }}

        .rewards-section {{
            background: #fff3cd;
            border: 1px solid #ffeaa7;
            border-radius: 8px;
            padding: 15px;
            margin: 20px;
            display: flex;
            flex-direction: column;
            gap: 10px;
        }}

        .rewards-title {{
            font-weight: 600;
            color: #856404;
            margin-bottom: 10px;
            display: flex;
            align-items: center;
            gap: 5px;
        }}

        .reward-item {{
            display: block;
            background: white;
            border: 1px solid #f1c40f;
            border-radius: 6px;
            padding: 8px 12px;
            margin: 3px 0;
            font-size: 0.9em;
        }}

        .no-results {{
            text-align: center;
            padding: 40px;
            color: #6c757d;
            font-size: 1.1em;
        }}

        .hidden {{
            display: none !important;
        }}

        /* Responsive Design */
        @media (max-width: 768px) {{
            .container {{
                margin: 10px;
                border-radius: 10px;
            }}

            .header {{
                padding: 20px;
            }}

            .header h1 {{
                font-size: 2em;
            }}

            .stats {{
                gap: 15px;
            }}

            .stat-item {{
                padding: 10px 15px;
            }}

            .search-container {{
                margin: 10px;
                padding: 15px;
            }}

            .search-options-grid {{
                grid-template-columns: 1fr;
            }}

            .quest-list {{
                padding: 20px;
            }}

            .quest-header {{
                padding: 15px;
            }}

            .quest-title {{
                font-size: 1.1em;
            }}

            .multilingual-content {{
                flex-direction: column;
                gap: 5px;
            }}

            .language-variant {{
                min-width: auto;
            }}

            .back-to-top {{
                bottom: 20px;
                right: 20px;
                width: 45px;
                height: 45px;
                font-size: 18px;
            }}
        }}

        /* Dark mode for TypeId=0 objectives */
        .objective-item.custom {{
            background: #f8f9fa;
            border-left: 4px solid #dc3545;
        }}

        .objective-type.custom {{
            background: #dc3545;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🗡️ RETRO Quest Database</h1>
            <p>Advanced Multilingual Quest Hierarchy Viewer with Text Search</p>
            <div class="stats">
                <div class="stat-item">
                    <span class="stat-number">{len(quest_data)}</span>
                    <span>Quests</span>
                </div>
                <div class="stat-item">
                    <span class="stat-number">{sum(len(q.get('Steps', [])) for q in quest_data)}</span>
                    <span>Steps</span>
                </div>
                <div class="stat-item">
                    <span class="stat-number">{sum(len(s.get('Objectives', [])) for q in quest_data for s in q.get('Steps', []))}</span>
                    <span>Objectives</span>
                </div>
                <div class="stat-item">
                    <span class="stat-number">{len(available_languages)}</span>
                    <span>Languages</span>
                </div>
            </div>
        </div>

        <div class="search-container">
            <div class="search-box">
                <input type="text" id="searchInput" placeholder="Search quests, steps, objectives, or items... (use * for wildcards)">
                
                <div class="search-controls">
                    <button id="applyFiltersBtn" class="apply-btn">
                        <span class="btn-text">Apply Search Filters</span>
                        <span class="btn-icon" style="display: none;">🔄</span>
                    </button>
                    <button id="toggleFiltersBtn" class="control-btn toggle-filters">
                        <span id="filterToggleText">Hide Filters</span>
                    </button>
                    <button id="collapseAllBtn" class="control-btn">Collapse All</button>
                    <button id="expandAllBtn" class="control-btn">Expand All</button>
                </div>
                
                <div class="search-options-grid" id="searchOptionsGrid">
                    <!-- Search Type Options -->
                    <div class="option-group">
                        <h4>🔍 Search Type</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="radio" name="searchType" value="text"><span class="option-text">📝 Search in Text</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="quest-name"><span class="option-text">🗡️ Search in Quest Name</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="step-name"><span class="option-text">👣 Search in Step Name</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="step-description"><span class="option-text">📖 Search in Step Description</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="objective"><span class="option-text">🎯 Search in Objectives</span></label>
                        </div>
                    </div>
                    
                    <!-- Text Search Options -->
                    <div class="option-group">
                        <h4>⚙️ Text Search Options</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="checkbox" id="exactMatch"><span class="option-text">🎯 Exact match</span></label>
                            <label class="option-row"><input type="checkbox" id="ignoreDiacritics"><span class="option-text">📝 Ignore diacritics (é=e)</span></label>
                            <label class="option-row"><input type="checkbox" id="useWildcards"><span class="option-text">⭐ Use wildcards (*)</span></label>
                        </div>
                    </div>
                    
                    <!-- Language Filters -->
                    <div class="option-group">
                        <h4>🔎 Search in Languages</h4>
                        <div class="option-table">"""
    
    # Add search language checkboxes
    lang_names = {'en': 'EN', 'fr': 'FR', 'es': 'ES', 'pt': 'PT'}
    lang_display_map = {lang: lang_names.get(lang, lang.upper()) for lang in available_languages}
    for lang in available_languages:
        lang_display = lang_display_map[lang]
        html_content += f'<label class="option-row"><input type="checkbox" name="searchLang" value="{lang}" checked><span class="option-text">{lang_display}</span></label>'
    
    html_content += f"""
                        </div>
                    </div>
                    
                    <!-- Display Options -->
                    <div class="option-group">
                        <h4>🖥️ Display Languages</h4>
                        <div class="option-table">"""
    
    # Add display language checkboxes - ALL checked by default
    for lang in available_languages:
        lang_display = lang_display_map[lang]
        html_content += f'<label class="option-row"><input type="checkbox" name="displayLang" value="{lang}" checked><span class="option-text">Show {lang_display}</span></label>'
    
    html_content += f"""
                        </div>
                    </div>
                    
                    <!-- Other Options -->
                    <div class="option-group">
                        <h4>🔧 Other Options</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="checkbox" id="showNullOnly"><span class="option-text">⚠️ Show NULL values only</span></label>
                            <label class="option-row"><input type="checkbox" id="enableCourierFont"><span class="option-text">📝 Use Courier New font</span></label>
                            <label class="option-row"><input type="checkbox" id="displayInRows"><span class="option-text">📋 Display languages in rows</span></label>
                            <label class="option-row language-highlight-row">
                                <span class="option-text">🌟 Highlight my language:</span>
                                <select id="highlightLanguage" class="language-dropdown">
                                    <option value="">None</option>"""
    
    # Add highlight language options
    for lang in available_languages:
        lang_display = lang_display_map[lang]
        html_content += f'<option value="{lang}">{lang_display}</option>'
    
    html_content += f"""
                                </select>
                            </label>
                        </div>
                    </div>
                </div>
            </div>
        </div>

        <div class="results-info" id="resultsInfo">
            Showing all {len(quest_data)} quests
        </div>

        <div class="quest-list display-columns" id="questList">
            <div class="language-header-row" id="languageHeaderRow">"""
    
    for lang in available_languages:
        lang_display = lang_display_map[lang]
        html_content += f'<div class="language-header" data-lang="{lang}" data-lang-display="{lang_display}">{lang_display}</div>'
    
    html_content += """
            </div>"""
    
    # Generate quest items
    for quest in quest_data:
        quest_id = quest.get('Id', 'Unknown')
        quest_name = quest.get('Name', {})
        category_id = quest.get('CategoryId', 0)
        steps = quest.get('Steps', [])
        rewards = quest.get('Rewards', {})
        
        # Create multilingual name structure with null value detection
        name_variants = []
        has_null_name = False
        for lang in available_languages:
            lang_display = lang_display_map[lang]
            name_value = quest_name.get(lang)
            if name_value is None or name_value == "" or name_value == "null":
                has_null_name = True
                name_variants.append(
                    f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text null-value">NULL</span></div>'
                )
            else:
                name_variants.append(
                    f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text">{name_value}</span></div>'
                )
        
        html_content += f"""
            <div class="quest-item" data-category="{category_id}" data-quest-id="{quest_id}" data-has-null="{str(has_null_name).lower()}">
                <div class="quest-header" onclick="toggleQuest(this)">
                    <div>
                        <div class="quest-title">
                            <div class="multilingual-content" data-field="name" data-quest-id="{quest_id}">
                                {''.join(name_variants)}
                            </div>
                        </div>
                        <div class="quest-meta">🗡️ ID: {quest_id} | Category: {category_id} | Steps: {len(steps)}</div>
                    </div>
                    <span class="expand-icon">▼</span>
                </div>
                <div class="quest-content expanded">"""
        
        # Add steps
        for step in steps:
            step_id = step.get('Id', 'Unknown')
            step_name = step.get('Name', {})
            step_description = step.get('Description', {})
            objectives = step.get('Objectives', [])
            
            # Create multilingual step name and description with null detection
            step_name_variants = []
            step_desc_variants = []
            has_null_step_name = False
            has_null_step_desc = False
            
            for lang in available_languages:
                lang_display = lang_display_map[lang]
                # Step name
                name_value = step_name.get(lang)
                if name_value is None or name_value == "" or name_value == "null":
                    has_null_step_name = True
                    step_name_variants.append(
                        f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text null-value">NULL</span></div>'
                    )
                else:
                    step_name_variants.append(
                        f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text">{name_value}</span></div>'
                    )
                
                # Step description
                desc_value = step_description.get(lang)
                if desc_value is None or desc_value == "" or desc_value == "null":
                    has_null_step_desc = True
                    step_desc_variants.append(
                        f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text null-value">NULL</span></div>'
                    )
                elif desc_value:
                    step_desc_variants.append(
                        f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text">{desc_value}</span></div>'
                    )
            
            html_content += f"""
                    <div class="step-item" data-has-null-name="{str(has_null_step_name).lower()}" data-has-null-desc="{str(has_null_step_desc).lower()}">
                        <div class="step-header" onclick="toggleStep(this)">
                            <div style="display: flex; align-items: center; gap: 15px; flex: 1;">
                                <span style="color: #6c757d; font-weight: 600; white-space: nowrap;">👣 ID: {step_id}</span>
                                <div data-field="step-name" data-step-id="{step_id}" style="flex: 1;">
                                    <div class="multilingual-content">
                                        {''.join(step_name_variants)}
                                    </div>
                                </div>
                            </div>
                            <span class="expand-icon">▼</span>
                        </div>
                        <div class="step-content expanded">"""
            
            if step_desc_variants:
                html_content += f"""
                            <div class="step-description" data-field="step-description" data-step-id="{step_id}">
                                <div class="multilingual-content">
                                    {''.join(step_desc_variants)}
                                </div>
                            </div>"""
            
            # Add objectives
            if objectives:
                html_content += f"""
                            <div class="objectives-section">
                                <div class="objectives-title">🎯 Objectives ({len(objectives)})</div>"""
                
                for obj in objectives:
                    obj_id = obj.get('Id', 'Unknown')
                    obj_type_id = obj.get('TypeId', 0)
                    obj_text = obj.get('Text', {})
                    obj_x = obj.get('X', 0)
                    obj_y = obj.get('Y', 0)
                    
                    # Create multilingual objective text with null detection
                    obj_text_variants = []
                    has_null_obj_text = False
                    for lang in available_languages:
                        lang_display = lang_display_map[lang]
                        text_value = obj_text.get(lang)
                        if text_value is None or text_value == "" or text_value == "null":
                            has_null_obj_text = True
                            obj_text_variants.append(
                                f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text null-value">NULL</span></div>'
                            )
                        else:
                            obj_text_variants.append(
                                f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text">{text_value}</span></div>'
                            )
                    
                    custom_class = 'custom' if obj_type_id == 0 else ''
                    coordinates = f"({obj_x}, {obj_y})" if obj_x != 0 or obj_y != 0 else ""
                    
                    html_content += f"""
                                <div class="objective-item {custom_class}" data-type-id="{obj_type_id}" data-has-null="{str(has_null_obj_text).lower()}">
                                    <span class="objective-type {custom_class}">🎯 ID: {obj_id}</span>
                                    <div data-field="objective-text" data-obj-id="{obj_id}">
                                        <div class="multilingual-content">
                                            {''.join(obj_text_variants)}
                                        </div>
                                    </div>
                                    {f'<span class="objective-coordinates">{coordinates}</span>' if coordinates else ''}
                                </div>"""
                
                html_content += "</div>"
            
            html_content += """
                        </div>
                    </div>"""
        
        # Add rewards with multilingual item names
        if (rewards.get('KamasReward', 0) > 0 or 
            rewards.get('ExperienceReward', 0) > 0 or 
            len(rewards.get('ItemsReward', [])) > 0 or 
            len(rewards.get('EmotesReward', [])) > 0):
            
            html_content += """
                    <div class="rewards-section">
                        <div class="rewards-title">🏆 Rewards</div>"""
            
            if rewards.get('KamasReward', 0) > 0:
                html_content += f'<div class="reward-item">💰 {rewards["KamasReward"]} Kamas</div>'
            
            if rewards.get('ExperienceReward', 0) > 0:
                html_content += f'<div class="reward-item">⭐ {rewards["ExperienceReward"]} XP</div>'
            
            # Items with multilingual names
            for item in rewards.get('ItemsReward', []):
                item_id = item["ItemId"]
                quantity = item["Quantity"]
                item_names = item.get("Name", {})
                
                if item_names:
                    # Create multilingual item display
                    item_name_variants = []
                    for lang in available_languages:
                        lang_display = lang_display_map[lang]
                        if item_names.get(lang):
                            item_name_variants.append(
                                f'<div class="language-variant visible lang-row" data-lang="{lang}" data-lang-display="{lang_display}"><span class="language-text">{item_names[lang]}</span></div>'
                            )
                    
                    html_content += f'''<div class="reward-item" title="Item ID: {item_id}">📦 <div class="multilingual-content">{''.join(item_name_variants)}</div> x{quantity}</div>'''
                else:
                    html_content += f'<div class="reward-item">📦 Item {item_id} x{quantity}</div>'
            
            for emote in rewards.get('EmotesReward', []):
                html_content += f'<div class="reward-item">😄 Emote {emote}</div>'
            
            html_content += "</div>"
        
        html_content += """
                </div>
            </div>"""
    
    html_content += f"""
        </div>
    </div>

    <!-- Back to top button -->
    <button class="back-to-top" id="backToTopBtn" onclick="scrollToTop()">
        ↑
    </button>

    <script>
        // Quest data for advanced filtering and language switching
        const questData = {json.dumps(quest_data, ensure_ascii=False)};
        const availableLanguages = {json.dumps(available_languages)};
        let visibleLanguages = {json.dumps(available_languages)};  // Show all languages by default
        let currentFilters = {{
            search: '',
            searchType: 'text', // Default to "Search in Text"
            exactMatch: false,
            ignoreDiacritics: false,
            useWildcards: false,
            searchLangs: {json.dumps(available_languages)},
            displayLangs: {json.dumps(available_languages)}, // Show all languages by default
            showNullOnly: false,
            highlightLanguage: '',
            displayInRows: false
        }};

        // Search-related variables
        const searchInput = document.getElementById('searchInput');
        const applyFiltersBtn = document.getElementById('applyFiltersBtn');
        const toggleFiltersBtn = document.getElementById('toggleFiltersBtn');
        const searchOptionsGrid = document.getElementById('searchOptionsGrid');
        const exactMatchCheckbox = document.getElementById('exactMatch');
        const ignoreDiacriticsCheckbox = document.getElementById('ignoreDiacritics');
        const useWildcardsCheckbox = document.getElementById('useWildcards');
        const showNullOnlyCheckbox = document.getElementById('showNullOnly');
        const enableCourierFontCheckbox = document.getElementById('enableCourierFont');
        const displayInRowsCheckbox = document.getElementById('displayInRows');
        const highlightLanguageDropdown = document.getElementById('highlightLanguage');
        const searchLangCheckboxes = document.querySelectorAll('input[name="searchLang"]');
        const displayLangCheckboxes = document.querySelectorAll('input[name="displayLang"]');
        const questItems = document.querySelectorAll('.quest-item');
        const backToTopBtn = document.getElementById('backToTopBtn');
        const questListElement = document.getElementById('questList');
        const languageHeaderRow = document.getElementById('languageHeaderRow');

        // Set default search type to "Search in Text"
        document.querySelector('input[name="searchType"][value="text"]').checked = true;

        function toggleQuest(header) {{
            const content = header.nextElementSibling;
            const icon = header.querySelector('.expand-icon');
            
            if (content.classList.contains('expanded')) {{
                content.classList.remove('expanded');
                icon.style.transform = 'rotate(0deg)';
            }} else {{
                content.classList.add('expanded');
                icon.style.transform = 'rotate(180deg)';
            }}
        }}

        function toggleStep(header) {{
            const content = header.nextElementSibling;
            const icon = header.querySelector('.expand-icon');
            
            if (content.classList.contains('expanded')) {{
                content.classList.remove('expanded');
                icon.style.transform = 'rotate(0deg)';
            }} else {{
                content.classList.add('expanded');
                icon.style.transform = 'rotate(180deg)';
            }}
        }}

        function toggleFilters() {{
            const container = document.querySelector('.search-container');
            const toggleText = document.getElementById('filterToggleText');
            
            if (container.classList.contains('filters-hidden')) {{
                container.classList.remove('filters-hidden');
                toggleText.textContent = 'Hide Filters';
            }} else {{
                container.classList.add('filters-hidden');
                toggleText.textContent = 'Show Filters';
            }}
        }}

        function scrollToTop() {{
            window.scrollTo({{
                top: 0,
                behavior: 'smooth'
            }});
        }}

        function updateDisplayLayout() {{
            const displayInRows = currentFilters.displayInRows;
            document.querySelectorAll('.multilingual-content').forEach(content => {{
                if (displayInRows) {{
                    content.classList.add('display-rows');
                }} else {{
                    content.classList.remove('display-rows');
                }}
            }});
            if (questListElement) {{
                questListElement.classList.toggle('display-rows', displayInRows);
                questListElement.classList.toggle('display-columns', !displayInRows);
            }}
            
            // Handle floating header visibility based on display mode
            const languageHeaderRow = document.getElementById('languageHeaderRow');
            if (languageHeaderRow) {{
                if (displayInRows) {{
                    languageHeaderRow.classList.remove('floating');
                }} else {{
                    // Re-trigger the floating logic
                    handleFloatingLanguageHeader();
                }}
            }}
        }}

        function removeDiacritics(str) {{
            return str.normalize("NFD").replace(/[\\u0300-\\u036f]/g, "");
        }}

        function wildcardToRegex(pattern) {{
            // Escape special regex characters except for *
            const escaped = pattern.replace(/[.*+?^${{}}()|[\\]\\\\]/g, '\\\\$&');
            // Replace * with .*
            return escaped.replace(/\\\\\\*/g, '.*');
        }}

        function highlightText(text, searchTerm, ignoreDiacritics, useWildcards, exactMatch) {{
            if (!searchTerm) return text;
            
            let processedText = text;
            let processedSearchTerm = searchTerm;
            
            if (ignoreDiacritics) {{
                processedText = removeDiacritics(processedText);
                processedSearchTerm = removeDiacritics(processedSearchTerm);
            }}
            
            let regex;
            if (useWildcards) {{
                const pattern = wildcardToRegex(processedSearchTerm);
                regex = new RegExp(`(${{pattern}})`, exactMatch ? 'g' : 'gi');
            }} else if (exactMatch) {{
                regex = new RegExp(`\\\\b(${{processedSearchTerm.replace(/[.*+?^${{}}()|[\\]\\\\]/g, '\\\\$&')}})\\\\b`, 'g');
            }} else {{
                regex = new RegExp(`(${{processedSearchTerm.replace(/[.*+?^${{}}()|[\\]\\\\]/g, '\\\\$&')}})`, 'gi');
            }}
            
            return processedText.replace(regex, '<span class="search-highlight">$1</span>');
        }}

        function updateLanguageDisplay() {{
            // Show/hide language variants based on display selection
            document.querySelectorAll('.language-variant').forEach(variant => {{
                const lang = variant.getAttribute('data-lang');
                if (currentFilters.displayLangs.includes(lang)) {{
                    variant.classList.add('visible');
                }} else {{
                    variant.classList.remove('visible');
                }}
            }});

            if (languageHeaderRow) {{
                let visibleHeaders = 0;
                languageHeaderRow.querySelectorAll('.language-header').forEach(header => {{
                    const lang = header.getAttribute('data-lang');
                    const shouldShow = currentFilters.displayLangs.includes(lang);
                    header.classList.toggle('hidden', !shouldShow);
                    if (shouldShow) {{
                        visibleHeaders += 1;
                    }}
                }});
                languageHeaderRow.classList.toggle('hidden', visibleHeaders === 0);
            }}
        }}

        function updateLanguageHighlighting() {{
            // Remove all existing highlighting
            document.querySelectorAll('.lang-row').forEach(row => {{
                availableLanguages.forEach(lang => {{
                    row.classList.remove(`highlighted-${{lang}}`);
                }});
            }});
            
            // Apply new highlighting
            if (currentFilters.highlightLanguage) {{
                document.querySelectorAll(`[data-lang="${{currentFilters.highlightLanguage}}"]`).forEach(element => {{
                    if (element.classList.contains('lang-row')) {{
                        element.classList.add(`highlighted-${{currentFilters.highlightLanguage}}`);
                    }}
                }});
            }}
        }}

        function toggleCourierFont() {{
            document.body.classList.toggle('courier-font', currentFilters.courierFont);
        }}

        function searchInText(searchTerm, searchType, searchLangs, ignoreDiacritics, useWildcards, exactMatch) {{
            const results = [];
            
            questData.forEach(quest => {{
                let questMatches = false;
                let matchDetails = [];
                
                // Search in quest names
                if (searchType === 'text' || searchType === 'quest-name') {{
                    if (quest.Name) {{
                        for (const lang of searchLangs) {{
                            const name = quest.Name[lang];
                            if (name && matchesSearchTerm(name, searchTerm, ignoreDiacritics, useWildcards, exactMatch)) {{
                                questMatches = true;
                                matchDetails.push({{ type: 'quest-name', lang, text: name }});
                            }}
                        }}
                    }}
                }}
                
                // Search in steps
                quest.Steps?.forEach(step => {{
                    // Step names
                    if (searchType === 'text' || searchType === 'step-name') {{
                        if (step.Name) {{
                            for (const lang of searchLangs) {{
                                const name = step.Name[lang];
                                if (name && matchesSearchTerm(name, searchTerm, ignoreDiacritics, useWildcards, exactMatch)) {{
                                    questMatches = true;
                                    matchDetails.push({{ type: 'step-name', lang, text: name }});
                                }}
                            }}
                        }}
                    }}
                    
                    // Step descriptions
                    if (searchType === 'text' || searchType === 'step-description') {{
                        if (step.Description) {{
                            for (const lang of searchLangs) {{
                                const desc = step.Description[lang];
                                if (desc && matchesSearchTerm(desc, searchTerm, ignoreDiacritics, useWildcards, exactMatch)) {{
                                    questMatches = true;
                                    matchDetails.push({{ type: 'step-description', lang, text: desc }});
                                }}
                            }}
                        }}
                    }}
                    
                    // Objectives
                    if (searchType === 'text' || searchType === 'objective') {{
                        step.Objectives?.forEach(obj => {{
                            if (obj.Text) {{
                                for (const lang of searchLangs) {{
                                    const text = obj.Text[lang];
                                    if (text && matchesSearchTerm(text, searchTerm, ignoreDiacritics, useWildcards, exactMatch)) {{
                                        questMatches = true;
                                        matchDetails.push({{ type: 'objective', lang, text }});
                                    }}
                                }}
                            }}
                        }});
                    }}
                }});
                
                if (questMatches) {{
                    results.push({{ questId: quest.Id, matches: matchDetails }});
                }}
            }});
            
            return results;
        }}

        function matchesSearchTerm(text, searchTerm, ignoreDiacritics, useWildcards, exactMatch) {{
            let processedText = text.toLowerCase();
            let processedSearchTerm = searchTerm.toLowerCase();
            
            if (ignoreDiacritics) {{
                processedText = removeDiacritics(processedText);
                processedSearchTerm = removeDiacritics(processedSearchTerm);
            }}
            
            if (useWildcards) {{
                const pattern = wildcardToRegex(processedSearchTerm);
                const regex = new RegExp(`^${{pattern}}$`, 'i');
                return regex.test(processedText);
            }} else if (exactMatch) {{
                return processedText === processedSearchTerm;
            }} else {{
                return processedText.includes(processedSearchTerm);
            }}
        }}

        function updateButtonState() {{
            const hasChanges = JSON.stringify(currentFilters) !== JSON.stringify(getFormState());
            applyFiltersBtn.classList.toggle('changed', hasChanges);
            if (hasChanges) {{
                applyFiltersBtn.querySelector('.btn-text').textContent = 'Apply Changes';
                applyFiltersBtn.querySelector('.btn-icon').style.display = 'inline';
            }} else {{
                applyFiltersBtn.querySelector('.btn-text').textContent = 'Apply Search Filters';
                applyFiltersBtn.querySelector('.btn-icon').style.display = 'none';
            }}
        }}

        function getFormState() {{
            return {{
                search: searchInput.value.toLowerCase().trim(),
                searchType: document.querySelector('input[name="searchType"]:checked').value,
                exactMatch: exactMatchCheckbox.checked,
                ignoreDiacritics: ignoreDiacriticsCheckbox.checked,
                useWildcards: useWildcardsCheckbox.checked,
                searchLangs: Array.from(searchLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value),
                displayLangs: Array.from(displayLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value),
                showNullOnly: showNullOnlyCheckbox.checked,
                highlightLanguage: highlightLanguageDropdown.value,
                courierFont: enableCourierFontCheckbox.checked,
                displayInRows: displayInRowsCheckbox.checked
            }};
        }}

        function performSearch() {{
            // Update current filters
            currentFilters = getFormState();
            
            // Hide filters after applying search
            const container = document.querySelector('.search-container');
            const toggleText = document.getElementById('filterToggleText');
            container.classList.add('filters-hidden');
            toggleText.textContent = 'Show Filters';
            
            // Apply font setting
            toggleCourierFont();
            
            // Apply display layout
            updateDisplayLayout();
            
            // Update language display and highlighting
            updateLanguageDisplay();
            updateLanguageHighlighting();
            
            // Clear previous highlights
            document.querySelectorAll('.search-highlight').forEach(highlight => {{
                highlight.outerHTML = highlight.innerHTML;
            }});
            
            let visibleCount = 0;
            
            questItems.forEach(item => {{
                let shouldShow = false;
                
                if (currentFilters.showNullOnly) {{
                    // Show quests that have null values
                    shouldShow = item.dataset.hasNull === 'true' ||
                                item.querySelector('[data-has-null="true"]') ||
                                item.querySelector('[data-has-null-name="true"]') ||
                                item.querySelector('[data-has-null-desc="true"]');
                }} else if (currentFilters.search === '') {{
                    shouldShow = true;
                }} else {{
                    // Perform text search
                    const questId = parseInt(item.dataset.questId);
                    const searchResults = searchInText(
                        currentFilters.search,
                        currentFilters.searchType,
                        currentFilters.searchLangs,
                        currentFilters.ignoreDiacritics,
                        currentFilters.useWildcards,
                        currentFilters.exactMatch
                    );
                    
                    shouldShow = searchResults.some(result => result.questId === questId);
                    
                    // Apply highlighting if there's a match
                    if (shouldShow && currentFilters.search) {{
                        item.querySelectorAll('.language-variant').forEach(variant => {{
                            const lang = variant.getAttribute('data-lang');
                            const textElement = variant.querySelector('.language-text');
                            if (!textElement || textElement.classList.contains('null-value')) {{
                                return;
                            }}
                            if (currentFilters.searchLangs.includes(lang)) {{
                                const textContent = textElement.textContent;
                                if (matchesSearchTerm(textContent, currentFilters.search, currentFilters.ignoreDiacritics, currentFilters.useWildcards, currentFilters.exactMatch)) {{
                                    textElement.innerHTML = highlightText(textContent, currentFilters.search, currentFilters.ignoreDiacritics, currentFilters.useWildcards, currentFilters.exactMatch);
                                }}
                            }}
                        }});
                    }}
                }}
                
                // Fix: Explicitly show/hide items
                if (shouldShow) {{
                    item.classList.remove('hidden');
                    item.style.display = 'block'; // Explicitly set to visible
                    visibleCount++;
                }} else {{
                    item.classList.add('hidden');
                }}
            }});

            // Update results info
            const resultsInfo = document.getElementById('resultsInfo');
            if (currentFilters.showNullOnly) {{
                resultsInfo.textContent = `Showing ${{visibleCount}} quests with NULL values`;
            }} else if (currentFilters.search) {{
                resultsInfo.textContent = `Found ${{visibleCount}} quests matching "${{currentFilters.search}}"`;
            }} else {{
                resultsInfo.textContent = `Showing all ${{visibleCount}} quests`;
            }}
            
            // Show no results message
            const existingNoResults = document.querySelector('.no-results');
            if (existingNoResults) existingNoResults.remove();
            
            if (visibleCount === 0) {{
                const questList = document.getElementById('questList');
                const noResultsDiv = document.createElement('div');
                noResultsDiv.className = 'no-results';
                noResultsDiv.innerHTML = '🔍 No quests found matching your criteria';
                questList.appendChild(noResultsDiv);
            }}
            
            updateButtonState();
        }}

        function collapseAll() {{
            document.querySelectorAll('.quest-content.expanded').forEach(content => {{
                content.classList.remove('expanded');
                const icon = content.previousElementSibling.querySelector('.expand-icon');
                icon.style.transform = 'rotate(0deg)';
            }});
            document.querySelectorAll('.step-content.expanded').forEach(content => {{
                content.classList.remove('expanded');
                const icon = content.previousElementSibling.querySelector('.expand-icon');
                icon.style.transform = 'rotate(0deg)';
            }});
        }}

        function expandAll() {{
            document.querySelectorAll('.quest-content').forEach(content => {{
                content.classList.add('expanded');
                const icon = content.previousElementSibling.querySelector('.expand-icon');
                icon.style.transform = 'rotate(180deg)';
            }});
            document.querySelectorAll('.step-content').forEach(content => {{
                content.classList.add('expanded');
                const icon = content.previousElementSibling.querySelector('.expand-icon');
                icon.style.transform = 'rotate(180deg)';
            }});
        }}

        // Back to top button functionality
        window.addEventListener('scroll', function() {{
            // Back to top button
            if (window.pageYOffset > 300) {{
                backToTopBtn.classList.add('show');
            }} else {{
                backToTopBtn.classList.remove('show');
            }}

            // Floating language header
            handleFloatingLanguageHeader();
        }});

        // Event listeners
        applyFiltersBtn.addEventListener('click', performSearch);
        toggleFiltersBtn.addEventListener('click', toggleFilters);
        
        document.getElementById('collapseAllBtn').addEventListener('click', collapseAll);
        document.getElementById('expandAllBtn').addEventListener('click', expandAll);
        
        // Monitor form changes for button state
        [searchInput, exactMatchCheckbox, ignoreDiacriticsCheckbox, useWildcardsCheckbox, 
         showNullOnlyCheckbox, enableCourierFontCheckbox, displayInRowsCheckbox, 
         highlightLanguageDropdown].forEach(element => {{
            element.addEventListener('input', updateButtonState);
            element.addEventListener('change', updateButtonState);
        }});
        
        document.querySelectorAll('input[name="searchType"]').forEach(radio => {{
            radio.addEventListener('change', updateButtonState);
        }});
        
        searchLangCheckboxes.forEach(checkbox => {{
            checkbox.addEventListener('change', updateButtonState);
        }});
        
        displayLangCheckboxes.forEach(checkbox => {{
            checkbox.addEventListener('change', updateButtonState);
        }});
        
        // Enter key to search
        searchInput.addEventListener('keypress', function(e) {{
            if (e.key === 'Enter') {{
                performSearch();
            }}
        }});

        // Floating language header functionality
        let originalLanguageHeaderPosition = null;

        function handleFloatingLanguageHeader() {{
            const languageHeaderRow = document.getElementById('languageHeaderRow');
            if (!languageHeaderRow) return;
            
            // Get the original position of the language header row
            if (originalLanguageHeaderPosition === null) {{
                const rect = languageHeaderRow.getBoundingClientRect();
                originalLanguageHeaderPosition = window.pageYOffset + rect.top;
            }}

            const currentScrollY = window.pageYOffset;
            const shouldFloat = currentScrollY > originalLanguageHeaderPosition + 100; // Add some buffer
            
            // Only show floating header in column mode, not row mode
            const isRowMode = currentFilters.displayInRows;

            if (shouldFloat && !isRowMode) {{
                languageHeaderRow.classList.add('floating');
            }} else {{
                languageHeaderRow.classList.remove('floating');
            }}
        }}

        // Initialize on page load
        document.addEventListener('DOMContentLoaded', function() {{
            updateLanguageDisplay();
            updateDisplayLayout();
            updateButtonState();
            updateLanguageHighlighting();
            
            // Set up initial floating header position
            setTimeout(() => {{
                handleFloatingLanguageHeader();
            }}, 100);
            
            // Expand all by default - icons are already rotated in CSS
            document.querySelectorAll('.expand-icon').forEach(icon => {{
                icon.style.transform = 'rotate(180deg)';
            }});
        }});
    </script>
</body>
</html>"""
    
    # Save HTML file
    html_filename = "Quest_Database_Viewer.html"
    with open(html_filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"\n✅ Advanced HTML Quest Database Viewer created: {html_filename}")
    print(f"📊 Database contains:")
    print(f"   • {len(quest_data)} quests")
    print(f"   • {sum(len(q.get('Steps', [])) for q in quest_data)} steps")
    print(f"   • {sum(len(s.get('Objectives', [])) for q in quest_data for s in q.get('Steps', []))} objectives")
    print(f"   • {len(available_languages)} languages: {', '.join(available_languages)}")
    
    # Count TypeId=0 objectives
    typeid_0_objectives = sum(1 for q in quest_data 
                             for s in q.get('Steps', []) 
                             for obj in s.get('Objectives', []) 
                             if obj.get('TypeId') == 0)
    
    # Count quests with rewards
    quests_with_rewards = sum(1 for q in quest_data if (
        q.get('Rewards', {}).get('KamasReward', 0) > 0 or
        q.get('Rewards', {}).get('ExperienceReward', 0) > 0 or
        len(q.get('Rewards', {}).get('ItemsReward', [])) > 0 or
        len(q.get('Rewards', {}).get('EmotesReward', [])) > 0
    ))
    
    # Count NULL values
    null_quest_names = sum(1 for q in quest_data for lang in available_languages 
                          if q.get('Name', {}).get(lang) in [None, "", "null"])
    null_step_names = sum(1 for q in quest_data for s in q.get('Steps', []) 
                         for lang in available_languages 
                         if s.get('Name', {}).get(lang) in [None, "", "null"])
    null_objectives = sum(1 for q in quest_data for s in q.get('Steps', []) 
                         for obj in s.get('Objectives', []) for lang in available_languages 
                         if obj.get('Text', {}).get(lang) in [None, "", "null"])
    
    print(f"   • {typeid_0_objectives} custom objectives (TypeId=0)")
    print(f"   • {quests_with_rewards} quests with rewards")
    print(f"   • {null_quest_names} NULL quest names")
    print(f"   • {null_step_names} NULL step names") 
    print(f"   • {null_objectives} NULL objective texts")
    
    print(f"\n🔍 Advanced Text Search Features:")
    print(f"   • 📝 'Search in Text' selected by default")
    print(f"   • 🎯 Exact match, wildcards (*), ignore diacritics")
    print(f"   • 🌍 Search in selected languages (independent from display)")
    print(f"   • 🖥️ Display all languages by default")
    print(f"   • 🌟 Highlight preferred language with dotted border")
    print(f"   • 📝 Courier New font option")
    print(f"   • ⚠️ Show NULL values only filter")
    print(f"   • 💡 Real-time search highlighting")
    print(f"   • 🔄 Collapse/Expand all functions")
    print(f"   • 👁️ Show/Hide filters button")
    print(f"   • 🎯 Objective IDs displayed with emoji prefixes")
    print(f"   • 📂 Expand all by default")
    print(f"   • ↑ Floating back-to-top arrow")
    print(f"   • 📊 Compact column layout for languages with shared headers")
    print(f"   • 📋 Optional row layout toggle")
    
    return html_filename

# Create the enhanced HTML viewer with all improvements
print("=== CREATING ENHANCED QUEST DATABASE VIEWER WITH UI IMPROVEMENTS ===\n")
html_file = create_quest_database_html()

if html_file:
    print(f"\n🎉 Success! Open '{html_file}' in your web browser to explore the quest database.")
    print("💡 New UI Improvements:")
    print("   • 📝 'Search in Text' is now the default search type")
    print("   • 📱 Better sticky positioning - header won't interfere with content")
    print("   • ↑ Floating back-to-top arrow appears when scrolling down")
    print("   • 📊 Languages displayed in compact columns for better space usage")
    print("   • 📋 Toggle option to switch between column and row layout")
    print("   • 🎨 Improved visual spacing and responsive design")
else:
    print("❌ Failed to create enhanced HTML viewer")

=== CREATING ENHANCED QUEST DATABASE VIEWER WITH UI IMPROVEMENTS ===

✓ Loaded 826 quests from multilingual database
✓ Available languages: ['en', 'es', 'fr', 'pt']
✓ Default language: en

✅ Advanced HTML Quest Database Viewer created: Quest_Database_Viewer.html
📊 Database contains:
   • 826 quests
   • 1235 steps
   • 2462 objectives
   • 4 languages: en, es, fr, pt
   • 506 custom objectives (TypeId=0)
   • 725 quests with rewards
   • 15 NULL quest names
   • 49 NULL step names
   • 0 NULL objective texts

🔍 Advanced Text Search Features:
   • 📝 'Search in Text' selected by default
   • 🎯 Exact match, wildcards (*), ignore diacritics
   • 🌍 Search in selected languages (independent from display)
   • 🖥️ Display all languages by default
   • 🌟 Highlight preferred language with dotted border
   • 📝 Courier New font option
   • ⚠️ Show NULL values only filter
   • 💡 Real-time search highlighting
   • 🔄 Collapse/Expand all functions
   • 👁️ Show/Hide filters button
   • 🎯 Objective IDs 

# 📊 Quest Database Schema Documentation

## 🗡️ Quest Schema Logic Tree (Multilingual Database)

```
📊 QUEST DATABASE SCHEMA (Multilingual)
│
├── 🗡️ QUEST (Root Entity)
│   ├── 🆔 Id: number
│   ├── 🌐 Name: {lang: string, ...} (multilingual)
│   ├── 🏷️ CategoryId: number
│   ├── 🏰 has_dungeon: boolean
│   ├── 📋 Steps: QuestStepData[] (0:many relationship)
│   └── 🎁 Rewards: QuestRewardData
│
├── 👣 QUEST STEP (Child of Quest)
│   ├── 🆔 Id: number
│   ├── 🌐 Name: {lang: string, ...} (multilingual)
│   ├── 🌐 Description: {lang: string, ...} (multilingual)
│   ├── 📊 OptimalLevel: number
│   └── 🎯 Objectives: QuestObjectiveData[] (0:many relationship)
│
├── 🎯 QUEST OBJECTIVE (Child of Step)
│   ├── 🆔 Id: number
│   ├── 🏷️ TypeId: number
│   ├── 🌐 Text: {lang: string, ...} (multilingual, resolved from template)
│   ├── 📍 X: number (coordinate)
│   ├── 📍 Y: number (coordinate)
│   └── 🔧 Parameters: number[] (numeric only)
│
└── 🎁 QUEST REWARDS (Part of Quest)
    ├── 💰 KamasReward: number
    ├── ⭐ ExperienceReward: number
    ├── 📦 ItemsReward: ItemReward[]
    │   ├── 🆔 ItemId: number
    │   ├── 🔢 Quantity: number
    │   └── 🌐 Name: {lang: string, ...} (multilingual)
    └── 😄 EmotesReward: number[]
```

## 📁 Enriched JSON Schema Tree (Structure File)

```
📁 quests_fr_1248_enriched.json
├── 🗂️ Q.q (Quests Section)
│   ├── 📝 value: "new Object()"
│   └── 📦 items: {}
│       └── 🔢 [quest_id]: {}
│           ├── 🏷️ c: number (CategoryId)
│           ├── 📋 s: [step_ids] (Step IDs array)
│           └── ⚙️ [other_metadata]: mixed
│
├── 🗂️ Q.s (Steps Section)  
│   ├── 📝 value: "new Object()"
│   └── 📦 items: {}
│       └── 🔢 [step_id]: {}
│           ├── 🎯 o: [objective_ids] (Objective IDs array)
│           ├── 🏰 d: number (Dungeon/Map ID)
│           ├── 📊 l: number (Level requirement)
│           ├── 🎁 r: [rewards_array] (Rewards)
│           └── ⚙️ [other_metadata]: mixed
│
├── 🗂️ Q.o (Objectives Section)
│   ├── 📝 value: "new Object()"
│   └── 📦 items: {}
│       └── 🔢 [objective_id]: {}
│           ├── 🏷️ t: number (TypeId)
│           ├── 📍 x: number (X coordinate)
│           ├── 📍 y: number (Y coordinate)
│           ├── 🔧 p: [numeric_params] (Non-string parameters only)
│           └── ⚙️ [other_metadata]: mixed
│
├── 🗂️ Q.t (Objective Types Templates)
│   ├── 📝 value: "new Object()"
│   └── 📦 items: {}
│       └── 🔢 [type_id]: null (All strings converted to null)
│
└── 📋 _metadata: {}
    ├── 🏗️ structure_info: {}
    ├── 📝 file_type: "quests"
    └── 🔍 parsing_pattern: "hierarchical"
```

## 🔗 Schema Relationships

### **1:Many Relationships**
```
🗡️ Quest (1) ──→ 👣 Steps (0:many)
   │
   └── Quest.Steps[] contains QuestStepData objects

👣 Step (1) ──→ 🎯 Objectives (0:many)
   │
   └── Step.Objectives[] contains QuestObjectiveData objects
```

### **ID-Based Relationships (Enriched File)**
```
📊 SCHEMA RELATIONSHIPS
│
├── 🔗 Primary Relationships (ID-based)
│   │
│   ├── Quest → Steps
│   │   │ Q.q[quest_id].s[] ──→ Q.s[step_id]
│   │   └── 📝 One quest can have multiple steps
│   │
│   ├── Step → Objectives  
│   │   │ Q.s[step_id].o[] ──→ Q.o[objective_id]
│   │   └── 📝 One step can have multiple objectives
│   │
│   └── Objective → Type Template
│       │ Q.o[objective_id].t ──→ Q.t[type_id]
│       └── 📝 One objective references one type (null in enriched)
```

## 🌍 Multilingual Text Resolution

```
🌍 MULTILINGUAL PATTERN
│
├── 📝 Static Text Fields
│   ├── Quest.Name: {en: "...", fr: "...", es: "...", pt: "..."}
│   ├── Step.Name: {en: "...", fr: "...", es: "...", pt: "..."}
│   └── Step.Description: {en: "...", fr: "...", es: "...", pt: "..."}
│
└── 🎯 Dynamic Objective Text (Template-based)
    ├── Objective.Text: {en: "...", fr: "...", es: "...", pt: "..."}
    ├── 🔍 Resolution Process:
    │   ├── 1. Get TypeId → Template from Q.t[TypeId]
    │   ├── 2. Get Parameters → Localized values per language
    │   ├── 3. Replace placeholders (#1, #2, etc.) with parameters
    │   └── 4. Return formatted text per language
    └── 📋 Special Cases:
        ├── TypeId = 0: Custom text (direct from parameters)
        └── TypeId ≠ 0: Template-based text with placeholder replacement
```

## 📈 Cardinality Summary

| Relationship | Cardinality | Description |
|--------------|-------------|-------------|
| **Quest → Steps** | `1:0..n` | One quest can have zero to many steps |
| **Step → Objectives** | `1:0..n` | One step can have zero to many objectives |
| **Quest → Rewards** | `1:1` | One quest has exactly one reward structure |
| **Rewards → Items** | `1:0..n` | One reward can contain zero to many items |

## 🔄 Data Transformation Flow

```
📊 DATA TRANSFORMATION FLOW

1️⃣ ActionScript Source
   │ quests_fr_1248.swf
   └── 📝 Contains: localized text + structure

2️⃣ Parsed JSON
   │ quests_fr_1248.json  
   └── 📝 Contains: mixed localized/non-localized

3️⃣ Custom Relationships
   │ api/custom/quests.json
   └── 📝 Contains: ID mappings + metadata

4️⃣ Enriched File (Non-localized)
   │ quests_fr_1248_enriched.json
   ├── ✅ Structure: IDs + relationships
   ├── ✅ Metadata: numeric values only
   ├── ❌ Localization: strings removed
   └── 📝 Purpose: language-independent base

5️⃣ Multilingual Database
   │ Quests_database_multilingual.json
   ├── 🏗️ Structure: from enriched file
   ├── 🌐 Text: from multiple language files
   └── 📝 Format: Cyberia QuestData structure
```

## 🧹 Localization Separation Strategy

```
🧹 LOCALIZATION SEPARATION
│
├── ❌ Removed from Enriched File
│   ├── 📝 Q.q[].n (quest names)
│   ├── 📝 Q.s[].n (step names)  
│   ├── 📝 Q.s[].d (step descriptions)
│   ├── 📝 Q.o[].p[strings] (string parameters)
│   └── 📝 Q.t[].* (all template strings → null)
│
└── ✅ Kept in Enriched File
    ├── 🔢 All numeric IDs
    ├── 🔗 All relationship arrays
    ├── 📊 All numeric metadata
    └── 🎁 All reward structures
```

## 🎯 Key Benefits

**🔄 Separation of Concerns:**
- **Structure** (enriched): IDs, relationships, numeric metadata
- **Localization** (language files): names, descriptions, templates

**🌍 Multilingual Support:**
- One structure file + multiple language files
- Efficient loading (structure once, text per language)
- Consistent IDs across all languages

**⚡ Performance:**
- No duplicate structural data across languages
- Smaller individual language files
- Faster multilingual switching

This architecture allows Cyberia to load the quest structure once and then overlay different languages without re-parsing the entire quest hierarchy.